In this notebook I will create a training table equivalent to ml_model_run_details

# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

### Function

#### expand_calc_features

In [2]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_list = []

    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)

    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')

    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)

    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    return result_df


#### expand_calc_features_robust

In [3]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_data = []

    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']

        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue

        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)

    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')

    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")

    return result_df

#### transform_data

In [4]:
# import pandas as pd
# import json
# import uuid
# from datetime import datetime
# from typing import List

# def transform_data(d1: pd.DataFrame, feature_column: List[str], a='demo_score', modelDisplayName = 'Cash_beta_trench1_Demo_backscore', subscription_name = 'sil_march 25 models'):
#     # Read the input CSV file
#     df = d1.copy()
    
#     # Create the output DataFrame with the required structure
#     output_data = []
    
#     for _, row in df.iterrows():
#         # Create the calcFeature JSON with all the feature columns
#         feature_columns = feature_column
        
#         calc_feature = {}
#         for col in feature_columns:
#             if col in row and pd.notna(row[col]):
#                 # Convert Timestamp objects to string
#                 if isinstance(row[col], pd.Timestamp):
#                     calc_feature[col] = row[col].isoformat()
#                 else:
#                     calc_feature[col] = row[col]
        
       
#         # Get current timestamp
#         current_time = datetime.now().isoformat()
        
#         # Create the output row
#         output_row = {
#             "customerId": row['customer_id'],
#             "digitalLoanAccountId": row['digitalLoanAccountId'],
#             "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
#             "prediction": row.get(a, 0),
#             "start_time": current_time,
#             "end_time": current_time,
#             "modelDisplayName":modelDisplayName,
#             "modelVersionId":"v1",
#             "subscription_name": subscription_name,
#             "message_id": str(uuid.uuid4()),  # Generate random UUID
#             "publish_time": current_time,
#             "attributes": "{}",  # Empty JSON object
#             "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
#         }
        
#         output_data.append(output_row)
    
#     # Create DataFrame from the output data
#     output_df = pd.DataFrame(output_data)
    
#     return output_df

# # Example usage:
# # transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# # transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [5]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_data(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v1",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],  # Device operating system
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


# transform_datav2

In [6]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_datav2(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v2",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],  # Device operating system
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


#### PSI Functions new

In [7]:
## Updated on 27-10-2025 - Modified for Training Period Baseline
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify

    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []

    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue

        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)

    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features (deciles with fallback) and categorical features (top 6 + others)
    based on the entire training period data.

    Parameters:
    -----------
    df : pd.DataFrame
        Full input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    train_period_df : pd.DataFrame
        Training period dataframe (June 2024 to March 2025)

    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    binning_info = {}

    # Create bins for numerical features with fallback strategy
    for feature in numerical_features:
        valid_data = train_period_df[feature].dropna()

        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue

        bins = None
        bin_count = None

        # Try 10 bins (deciles)
        try:
            test_bins = np.percentile(valid_data, np.arange(0, 101, 10))
            test_bins = np.unique(test_bins)
            if len(test_bins) >= 11:  # 11 edges = 10 bins
                bins = test_bins
                bin_count = 10
        except Exception as e:
            pass

        # If 10 bins not possible, try 5 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, np.arange(0, 101, 20))
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 6:  # 6 edges = 5 bins
                    bins = test_bins
                    bin_count = 5
            except Exception as e:
                pass

        # If 5 bins not possible, try 3 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, [0, 33.33, 66.67, 100])
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 4:  # 4 edges = 3 bins
                    bins = test_bins
                    bin_count = 3
            except Exception as e:
                pass

        # If still no bins possible, use equal distance bins of 5
        if bins is None:
            print(f"Warning: Feature '{feature}' has insufficient variance - cannot create standard bins")
            print(f"Feature '{feature}': Using equal distance bins of 5")

            min_val = valid_data.min()
            max_val = valid_data.max()

            # Create 5 equal distance bins
            bins = np.linspace(min_val, max_val, 6)  # 6 edges = 5 bins
            bins = np.unique(bins)
            bin_count = len(bins) - 1

            # If all values are the same, add slight buffer
            if bin_count == 1:
                bins = np.array([min_val - 0.1, min_val, min_val + 0.1])
                bin_count = 2
                print(f"Feature '{feature}': Constant value ({min_val}). Created 2 equal distance bins with buffer")

        # Add infinity edges to capture all values
        bins = bins.copy()
        bins[0] = -np.inf
        bins[-1] = np.inf

        print(f"Feature '{feature}': Created {bin_count} bins")

        # Create bin ranges dictionary
        bin_ranges = {}
        for i in range(len(bins)-1):
            bin_name = f"Bin_{i+1}"
            bin_ranges[bin_name] = {
                'min': bins[i],
                'max': bins[i+1],
                'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
            }

        binning_info[feature] = {
            'type': 'numerical',
            'bins': bins,
            'bin_ranges': bin_ranges,
            'bin_count': bin_count
        }

    # Create bins for categorical features (top 6 + others) using training period
    for feature in categorical_features:
        value_counts = train_period_df[feature].value_counts()
        unique_categories = value_counts.index.tolist()
        print(f"Unique categories: {unique_categories}")

        if len(unique_categories) <= 6:
            # Treat each category as a separate bin
            top_categories = unique_categories
        else:
            # Use top 6 categories only
            top_categories = value_counts.nlargest(6).index.tolist()

        print(f"Top categories for feature '{feature}': {top_categories}")

        binning_info[feature] = {
                'type': 'categorical',
                'top_categories': top_categories,
                'bin_ranges': {}  # No ranges for categorical
            }

    return binning_info


def apply_binning(df: pd.DataFrame,
                  feature: str,
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature

    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)

        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]

        binned = pd.cut(df[feature],
                       bins=bins,
                       labels=labels,
                       include_lowest=True,
                       duplicates='drop')

        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'

        return binned

    else:  # categorical
        top_cats = binning_info['top_categories']

        # Convert to string for consistent comparison
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)

        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')

        # Convert top_cats to strings for comparison
        top_cats_str = [str(cat) for cat in top_cats]

        # Apply binning logic: use category name if in top_cats, else 'Others' (except for Missing)
        binned = feature_data.apply(lambda x: x if x in top_cats_str else ('Others' if x != 'Missing' else 'Missing'))

        return binned


def calculate_psi(expected_pct: pd.Series,
                  actual_pct: pd.Series,
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.

    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)

    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)

    # Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)

    # Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()

    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))

    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period (June 2024 to March 2025)
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column (identifies train period)
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []

    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(f"Feature binned {df[f'{feature}_binned']}")
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate PSI for each test month
        for month in test_months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(train_baseline, actual_dist)

            # Calculate average percentages across all bins
            expected_avg_pct = train_baseline.mean() * 100
            actual_avg_pct = actual_dist.mean() * 100

            # # Count distinct accounts for segment
            # base_segment_count = train_segment[account_id_col].nunique()
            # actual_segment_count = actual_segment[account_id_col].nunique()


            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                'Current_Month': month,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value
            })

    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(train_baseline, actual_dist)

                    # Calculate average percentages across all bins
                    expected_avg_pct = train_baseline.mean() * 100
                    actual_avg_pct = actual_dist.mean() * 100

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                        'Current_Month': month,
                        'Base_Count': base_segment_count,
                        'Actual_Count': actual_segment_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value
                    })

    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month',
                            data_selection_col: str = 'Data_selection',
                            account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature comparing training period
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []
    epsilon = 0.0001

    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(df[f'{feature}_binned'])

        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate bin-level PSI for each test month
        for month in test_months:
            month_data = df[df[month_col] == month]
            actual_dist = month_data[f'{feature}_binned'].value_counts(normalize=True)

            # Count distinct accounts
            base_count = df[df[data_selection_col] == 'Train'][account_id_col].nunique()
            actual_count = month_data[account_id_col].nunique()

            # Get all bins
            all_bins = train_baseline.index.union(actual_dist.index)

            for bin_name in all_bins:
                # Simplified epsilon logic - no redundancy
                expected_pct = train_baseline.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)

                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct

                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name

                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                    'Current_Month': month,
                    'Base_Count': base_count,
                    'Actual_Count': actual_count,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })

    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate bin-level PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    # Get all bins
                    all_bins = train_baseline.index.union(actual_dist.index)

                    for bin_name in all_bins:
                        # Simplified epsilon logic - no redundancy
                        expected_pct = train_baseline.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)

                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct

                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name

                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                            'Current_Month': month,
                            'Base_Count': base_segment_count,
                            'Actual_Count': actual_segment_count,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })

    return pd.DataFrame(results)

# SIL V1

## SIL

#### Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

##### 'Alpha - CIC-SIL-Model'

In [8]:
sq = """select distinct
    r.customerId customer_id ,
    r.digitalLoanAccountId,
    r.cic_score,
    r.cic_Personal_Loans_granted_contracts_amt_24M,
    r.cic_days_since_last_inquiry, 
    r.cic_cnt_active_contracts,
    r.cic_vel_contract_nongranted_cnt_12on24,
    r.cic_max_amt_granted_24M, 
    r.cic_zero_non_granted_ever_flag,
    r.cic_tot_active_contracts_util,
    r.cic_vel_contract_granted_amt_12on24,
    r.cic_zero_granted_ever_flag,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from risk_mart.sil_risk_ds_master_20230101_20250309_v2 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where cic_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) <= '2025-01-31'
;"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID fe002394-5305-4752-b17c-9100deb4d048 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (184484, 13)


In [9]:
feature_column = ['cic_Personal_Loans_granted_contracts_amt_24M',
       'cic_days_since_last_inquiry', 'cic_cnt_active_contracts',
       'cic_vel_contract_nongranted_cnt_12on24', 'cic_max_amt_granted_24M',
       'cic_zero_non_granted_ever_flag', 'cic_tot_active_contracts_util',
       'cic_vel_contract_granted_amt_12on24', 'cic_zero_granted_ever_flag']

In [10]:
dfd = transform_data(data, feature_column, a='cic_score', modelDisplayName='Alpha - CIC-SIL-Model') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3046561,4148b44c-971d-4216-9b66-28048905f424,282c8e22-d62c-49ce-afc6-3ca1b2815900,0.126142,2025-12-23T22:16:16.225752,2025-12-23T22:16:16.225752,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 1743.0, ""cic_z...",sil_march 25 models,dd3bf7ab-cf51-43ca-a10f-89b3459f3e99,2025-12-23T22:16:16.225752,{},,android
1,3185300,f724d608-64e1-4e9d-9a87-454c216b2b14,acea8a6d-3b52-4b8c-b132-965469dc59cb,0.121053,2025-12-23T22:16:16.226307,2025-12-23T22:16:16.226307,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 361.0, ""cic_cn...",sil_march 25 models,4f5c2f71-f3d0-4d20-8840-60b8ca296e0a,2025-12-23T22:16:16.226307,{},,android
2,3069732,3f3cf085-2521-4451-b960-b843afaf014f,2903479d-1de8-4306-94bc-ab2cddaecc36,0.202277,2025-12-23T22:16:16.227522,2025-12-23T22:16:16.227522,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 0.0, ""cic_vel_...",sil_march 25 models,0fce51e6-da17-49b7-bea3-4230884a11ff,2025-12-23T22:16:16.227522,{},,android
3,3126903,db5ff269-fbf8-4566-9c47-649faf84d279,b6818e8f-325f-45e7-bcf5-9b5de3c6955d,0.092074,2025-12-23T22:16:16.227522,2025-12-23T22:16:16.227522,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 773.0, ""cic_cn...",sil_march 25 models,f83b5b88-2a05-4148-90e1-b8d70ea121fb,2025-12-23T22:16:16.227522,{},,android
4,2848247,7cc09a6a-4d06-402d-9644-caafe84ad192,6fe21dcf-8920-414a-a7a6-b84974b69db8,0.199273,2025-12-23T22:16:16.228095,2025-12-23T22:16:16.228095,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 0.0, ""cic_vel_...",sil_march 25 models,bcbdad06-4800-4dd7-a6f2-5fd2b940489e,2025-12-23T22:16:16.228095,{},,android


In [11]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=1bdcabe7-2001-4a3c-9006-a6456dc04f52>

##### Alpha Sil Stack Model 

In [12]:
sq = """ 
select distinct 
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_stack_score,
r.beta_demo_score,
r.cic_score,
r.apps_score,
r.credo_gen_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where alpha_stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) <= '2025-01-31'
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID aee028e0-2a3e-4f22-9167-82e326fb6045 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (184484, 8)


In [13]:
data.columns
data.rename(columns={'beta_demo_score':'sb_demo_score', 'cic_score':'s_cic_score', 
                    'apps_score':'s_apps_score', 'credo_gen_score':'s_credo_score'}, inplace = True)
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'alpha_stack_score',
       'sb_demo_score', 's_cic_score', 's_apps_score', 's_credo_score',
       'osType'],
      dtype='object')

In [14]:
feature_column = ['sb_demo_score',
       's_cic_score', 's_apps_score',
       's_credo_score']

dfd = transform_data(data, feature_column, a='alpha_stack_score', modelDisplayName='Alpha - StackingModel') 
dfd.head()


,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2535150,ad7ea617-71b6-40c7-abe6-fa25f46c3718,40ea07d7-83cd-4294-a9a1-1ae11f4f9459,0.108383,2025-12-23T22:16:47.488353,2025-12-23T22:16:47.488353,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.25245539172019843, ""s_cic_...",sil_march 25 models,7ddfbc76-67bc-46a8-a580-1d38a9d727fa,2025-12-23T22:16:47.488353,{},,android
1,2429167,654a9814-0b4c-4ba3-904f-786f40d1db7b,56ac088f-fc26-48e7-b916-b25ea39be84e,0.126910,2025-12-23T22:16:47.488353,2025-12-23T22:16:47.488353,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.11969479562898067, ""s_cic_...",sil_march 25 models,cf221db7-3b9e-4175-bf84-b0fca4fcf728,2025-12-23T22:16:47.488353,{},,android
2,3109385,f5abef55-78c5-4bc7-a164-1c183ce0aaa2,4d950dc2-a50d-4c11-b87a-973a3bb1ad91,0.050433,2025-12-23T22:16:47.489355,2025-12-23T22:16:47.489355,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.06146992924368682, ""s_cic_...",sil_march 25 models,f8ca2146-7f00-4fb1-a6e6-dcddd128e519,2025-12-23T22:16:47.489355,{},,android
3,3092274,20139a47-12a3-4912-a82f-e837f3214ba5,5fcc8175-2386-4cea-b2a5-9b491338491a,0.277837,2025-12-23T22:16:47.489355,2025-12-23T22:16:47.489355,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.11042569726750293, ""s_cic_...",sil_march 25 models,3256f946-51d0-4cf2-987f-d042bacb953e,2025-12-23T22:16:47.489355,{},,android
4,2760690,501e2598-4db1-41c7-a917-682f8fba1a95,27eca6db-8bb1-401f-831c-5f8666593175,0.274645,2025-12-23T22:16:47.489355,2025-12-23T22:16:47.489355,Alpha - StackingModel,v1,"{""sb_demo_score"": 0.2125766367312137, ""s_cic_s...",sil_march 25 models,3cfca947-377b-4a20-80f7-1fd5a2992e1d,2025-12-23T22:16:47.489355,{},,android


In [15]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e0cb2cd6-599b-4edc-a87a-27ac5a2768ac>

##### Beta Sil App Score

In [16]:
sq = """ 
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.apps_score,
r.app_cnt_absence_tag_30d,
r.app_cnt_absence_tag_90d ,
r.app_cnt_business_ever ,
r.app_cnt_competitors_30d ,
r.app_cnt_competitors_90d ,
r.app_cnt_education_ever ,
r.app_cnt_finance_7d ,
r.app_cnt_finance_90d ,
r.app_cnt_music_and_audio_ever ,
r.app_cnt_payday_90d ,
r.app_cnt_rated_for_3plus_ever ,
r.app_cnt_travel_and_local_ever ,
r.app_first_competitors_install_to_apply_days ,
r.app_first_payday_install_to_apply_days ,
r.app_median_time_bw_installed_mins_30d ,
r.app_vel_finance_30_over_365 ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) <= '2024-06-30'
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 4b360c88-e1ba-46ad-a599-9d94caaa91b5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (116033, 20)


In [17]:
feature_column = ['app_cnt_rated_for_3plus_ever',
       'app_cnt_education_ever', 'app_cnt_business_ever',
       'app_cnt_music_and_audio_ever',
       'app_cnt_travel_and_local_ever', 'app_cnt_finance_7d',
       'app_cnt_competitors_30d', 'app_cnt_absence_tag_30d',
        'app_cnt_absence_tag_90d',
       'app_cnt_finance_90d', 'app_cnt_competitors_90d',
       'app_cnt_payday_90d',
       'app_median_time_bw_installed_mins_30d',
       'app_first_competitors_install_to_apply_days',
       'app_first_payday_install_to_apply_days',
       'app_vel_finance_30_over_365']

dfd = transform_data(data, feature_column, a='apps_score', modelDisplayName='Beta - AppsScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2318601,3e8a3bd8-0a7e-4193-bc32-23ab4bb0f436,9bf1f4ba-a4b3-4db2-a02c-3454ba39174f,0.507398,2025-12-23T22:17:14.879372,2025-12-23T22:17:14.879372,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 41.0, ""app_cn...",sil_march 25 models,fb0db762-668c-401f-9fad-a0689c4e9604,2025-12-23T22:17:14.879372,{},,android
1,2548491,e6a9ee13-4f0a-48b8-a701-e211a2945461,521cbed2-9444-4c5e-8f7c-9a73fe926f64,0.637309,2025-12-23T22:17:14.880373,2025-12-23T22:17:14.880373,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 56.0, ""app_cn...",sil_march 25 models,70f4c5fe-c221-4160-a5d5-8d70a99c13c1,2025-12-23T22:17:14.880373,{},,android
2,2235732,7966be56-f2b0-47a9-ad73-b5387642a807,0ca83b6e-9883-4e4c-9603-e060911cb96a,0.564166,2025-12-23T22:17:14.880373,2025-12-23T22:17:14.880373,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 39.0, ""app_cn...",sil_march 25 models,80f8b307-5d10-4d46-822f-57045595099e,2025-12-23T22:17:14.880373,{},,android
3,2572056,3bab27e8-9400-4e3f-b43e-ce05c869432e,ef5a40ed-f87d-46ba-8480-a255d16f053b,0.413147,2025-12-23T22:17:14.880373,2025-12-23T22:17:14.880373,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 86.0, ""app_cn...",sil_march 25 models,37758446-52f6-42b1-b553-2c82b91bf370,2025-12-23T22:17:14.880373,{},,android
4,2405015,4c291f8f-9685-4100-ba39-91a6e06d61ec,2ab9685d-52f6-48be-b08c-f788c31f32f8,0.440001,2025-12-23T22:17:14.880373,2025-12-23T22:17:14.880373,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 39.0, ""app_cn...",sil_march 25 models,431bdf48-7dfd-4654-a39c-e8e36170c8f1,2025-12-23T22:17:14.880373,{},,android


In [18]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ebbf9132-b637-4d89-9a0f-85754f782c8b>

##### Beta SIL Demo Score

In [19]:
sq = """
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_demo_score,
r.beta_de_ln_vas_opted_flag ,
r.beta_de_ln_doc_type_rolled ,
r.beta_de_ln_marital_status ,
r.beta_de_ln_age_bin ,
r.beta_de_ln_province_bin ,
r.beta_de_ln_ref2_type ,
r.beta_de_ln_education_level ,
r.beta_de_ln_ref1_type ,
r.beta_de_ln_industry_new_bin ,
r.beta_de_ln_appln_day_of_week ,
r.beta_de_onb_name_email_match_score ,
r.beta_de_ln_employment_type_new_bin ,
r.beta_de_ln_telconame ,
r.beta_de_time_bw_onb_loan_appln_mins ,
r.beta_de_ln_source_of_funds_new_bin ,
r.beta_de_ln_brand_bin ,
r.beta_de_ln_email_primary_domain ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_demo_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) <= '2025-01-31'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID cd09f2e9-7659-4dda-9a4b-8896e13e8212 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (321067, 21)


In [20]:
feature_column = ['beta_de_ln_vas_opted_flag',
       'beta_de_ln_doc_type_rolled', 'beta_de_ln_marital_status',
       'beta_de_ln_age_bin', 'beta_de_ln_province_bin',
       'beta_de_ln_ref2_type', 'beta_de_ln_education_level',
       'beta_de_ln_ref1_type', 'beta_de_ln_industry_new_bin',
       'beta_de_ln_appln_day_of_week',
       'beta_de_onb_name_email_match_score',
       'beta_de_ln_employment_type_new_bin', 'beta_de_ln_telconame',
       'beta_de_time_bw_onb_loan_appln_mins',
       'beta_de_ln_source_of_funds_new_bin', 'beta_de_ln_brand_bin',
       'beta_de_ln_email_primary_domain']

dfd = transform_data(data, feature_column, a='beta_demo_score', modelDisplayName='Beta - DemoScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2935821,2efc28f7-e876-4a19-b158-e82d8d133fb1,47f71cb9-7d7e-48d3-a56c-e2ac704398e8,0.264842,2025-12-23T22:17:47.093552,2025-12-23T22:17:47.093552,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,121fade8-2774-4cb3-a5a1-40e081a391bf,2025-12-23T22:17:47.093552,{},,android
1,1394227,a2a9a53b-c3ec-4c3d-aaa6-34ceb61fafcd,bbea10b5-68ce-4541-afad-b6000975e1a5,0.392506,2025-12-23T22:17:47.093552,2025-12-23T22:17:47.093552,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,edc3832e-2c5a-4f5f-a3bc-28cbb05ef4fe,2025-12-23T22:17:47.093552,{},,ios
2,2896488,7a3dde41-0e35-4f42-8812-ba581ce92bcd,eed644ca-0f52-4315-bac3-a1d5c5d34d03,0.105452,2025-12-23T22:17:47.093552,2025-12-23T22:17:47.093552,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",sil_march 25 models,6965200f-4d83-41fe-b9b0-e2575b00a8f0,2025-12-23T22:17:47.093552,{},,android
3,2328687,36775240-08d1-4afa-8fd5-55bf47e9796f,8d57bb0e-ba10-458a-bef0-0cf2ab0efd18,0.387968,2025-12-23T22:17:47.093552,2025-12-23T22:17:47.093552,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,4eed6156-54e1-4a29-bd32-a9e739aafb6e,2025-12-23T22:17:47.093552,{},,android
4,2900143,aea271d0-efb1-4c92-bff2-55de01eeb155,d3aa10e0-0f2f-4549-ba86-6abc6fead546,0.082794,2025-12-23T22:17:47.093552,2025-12-23T22:17:47.093552,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",sil_march 25 models,1992a959-8152-4823-a954-f4df0f6be897,2025-12-23T22:17:47.093552,{},,android


In [21]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=acf7ce81-82ec-47dc-a67d-31472469a0f6>

##### Beta SIL STACK Score Model

In [22]:
sq = """ 
select  distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_stack_score,
r.apps_score,
r.credo_gen_score,
r.beta_demo_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) <= '2025-01-31'
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID b141c37d-b4b2-458a-89e6-0a63ca4d4915 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (321067, 7)


In [23]:
feature_column = ['apps_score', 'credo_gen_score', 'beta_demo_score']
dfd = transform_data(data, feature_column, a='beta_stack_score', modelDisplayName='Beta - StackScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2597915,f7d6fc50-8b5a-4e7f-8240-5144e2e8feba,35802b6a-3fed-4bfd-b370-1a1892632258,0.027061,2025-12-23T22:18:59.756982,2025-12-23T22:18:59.756982,Beta - StackScoreModel,v1,"{""apps_score"": 0.48427189444898167, ""credo_gen...",sil_march 25 models,d2904c36-f11f-495b-9e5d-59e1ee481926,2025-12-23T22:18:59.756982,{},,android
1,2699160,db2b2c6d-ddd0-4ada-82d4-a82b60bcf011,0970c6ef-f072-452c-8d7a-a622d3e2b44e,0.195342,2025-12-23T22:18:59.756982,2025-12-23T22:18:59.756982,Beta - StackScoreModel,v1,"{""apps_score"": 0.6144126474211813, ""credo_gen_...",sil_march 25 models,7901dd56-23b2-4495-bcd0-0b3760d15ee8,2025-12-23T22:18:59.756982,{},,android
2,2520819,e54ecfc8-f104-4164-b295-4d85381fcac7,180dc011-6531-48a7-9951-c1da0727cdb1,0.129511,2025-12-23T22:18:59.756982,2025-12-23T22:18:59.756982,Beta - StackScoreModel,v1,"{""apps_score"": 0.47128050886272654, ""credo_gen...",sil_march 25 models,9dfe0633-2656-4c4b-8ff3-40d43036b28b,2025-12-23T22:18:59.756982,{},,android
3,2223027,53365edb-7625-4027-90ef-89c8eed9e8dc,38756b0b-ddfe-44b6-885e-aa11c15ecab7,0.324124,2025-12-23T22:18:59.756982,2025-12-23T22:18:59.756982,Beta - StackScoreModel,v1,"{""apps_score"": 0.6591773297035802, ""credo_gen_...",sil_march 25 models,1a6f3805-af44-4824-9731-e0b51ec05cdc,2025-12-23T22:18:59.756982,{},,android
4,3211941,c4b34db7-c216-45f8-a0d7-30a99bdd8daa,544c4cd2-f4c3-477b-a185-a5a999ef5d1d,0.103842,2025-12-23T22:18:59.756982,2025-12-23T22:18:59.756982,Beta - StackScoreModel,v1,"{""apps_score"": 0.543560943902095, ""credo_gen_s...",sil_march 25 models,cc174fa1-b5c1-4e56-b562-de5adfb5581a,2025-12-23T22:18:59.756982,{},,android


In [24]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c255ffc4-77c0-4e27-86d5-3a5f1331f00f>

##### Alpha  - IncomeEstimationModel

In [25]:
sq = """  
Select 
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_estimated_income,
r.inc_alpha_cic_credit_avg_credit_limit,
r.inc_alpha_cic_max_active_contracts_amt,
r.inc_alpha_ln_company_name,
r.inc_alpha_ln_age,
r.inc_alpha_doc_type_rolled,
r.inc_alpha_ln_brand,
r.inc_alpha_ln_city,
r.inc_alpha_ln_cnt_dependents,
r.inc_alpha_ln_education_level,
r.inc_alpha_ln_employment_type_new,
r.inc_alpha_ln_gender,
r.inc_alpha_ln_industry_new,
r.inc_alpha_ln_loan_prod_type,
r.inc_alpha_ln_marital_status_new,
r.inc_alpha_ln_nature_of_work_new,
r.inc_alpha_ln_osversion_bin,
r.inc_alpha_ln_purpose,
r.inc_alpha_ln_source_of_funds_new,
r.inc_alpha_loan_monthly_income,
r.inc_alpha_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID a726a93a-2da6-441a-bb72-b4e7aba88cd4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 24)


In [26]:
feature_column = ['inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped']
dfd = transform_data(data, feature_column, a='alpha_estimated_income', modelDisplayName='Alpha  - IncomeEstimationModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3263610,d3202c3d-3858-48e3-b11f-2c54cc7817f9,7e6aa03c-6ece-47fd-8a0d-b9c3b8d95c45,8193.496378,2025-12-23T22:19:48.883713,2025-12-23T22:19:48.883713,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,4e870248-8191-40da-8a1f-6e04a929c436,2025-12-23T22:19:48.883713,{},,android
1,2674924,cc66090d-abce-4724-b34f-ca48e81d7c3b,c06f0a54-a68b-4b1d-905c-d940288d8c47,8923.717325,2025-12-23T22:19:48.883713,2025-12-23T22:19:48.883713,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,11e71cf1-7f4b-4764-be9c-608b7bd72de2,2025-12-23T22:19:48.883713,{},,android
2,2598077,518b07bb-5e2a-4221-9de9-ecf860c4d1ee,42b9d511-2059-42ef-bb1c-3414c43ea145,7764.905986,2025-12-23T22:19:48.883713,2025-12-23T22:19:48.883713,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,88964736-d6f5-420e-89f1-c341663c2149,2025-12-23T22:19:48.883713,{},,android
3,2447064,7d53acff-4665-456b-96a0-da717d938ca1,841de09b-4551-4470-9247-e88547c364b1,8688.720235,2025-12-23T22:19:48.884718,2025-12-23T22:19:48.884718,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,9be2330e-0660-484c-969e-91321e325320,2025-12-23T22:19:48.884718,{},,android
4,2482660,16dffffb-fc9c-4e09-b8b4-c1844e601aa8,ac5e8fb1-7144-401c-a7b8-7ff0678ac329,30884.574594,2025-12-23T22:19:48.884718,2025-12-23T22:19:48.884718,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 2000...",sil_march 25 models,9627954a-09e9-43b5-8d9b-7ead060d36da,2025-12-23T22:19:48.884718,{},,ios


In [27]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3fc41559-db0e-4e8e-92ef-e6a0427948b8>

##### Beta - IncomeEstimationModelm

In [28]:
sq = """ 
select
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_estimated_income,
r.inc_beta_ln_loan_type,
r.inc_beta_ln_education_level,
r.inc_beta_ln_employment_type_new,
r.inc_beta_ln_industry_new,
r.inc_beta_ln_age,
r.inc_beta_ln_brand,
r.inc_beta_ln_city,
r.inc_beta_ln_purpose,
r.inc_beta_ln_osversion_bin,
r.inc_beta_ln_postal_code,
r.inc_beta_ln_gender,
r.inc_beta_ln_doc_type_rolled,
r.inc_beta_ln_cnt_dependents,
r.inc_beta_ln_source_of_funds_new,
r.inc_beta_ln_marital_status_new,
r.inc_beta_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""

# data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
data = client.query(sq).result().to_arrow().to_pandas()
print(f"The shape of the dataframe is:\t {data.shape}")

The shape of the dataframe is:	 (349207, 20)


In [29]:
feature_column = ['inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

dfd = transform_data(data, feature_column, a='beta_estimated_income', modelDisplayName='Beta - IncomeEstimationModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2699160,db2b2c6d-ddd0-4ada-82d4-a82b60bcf011,554603ee-8492-4de5-881a-f28cc711dbe1,7448.291127,2025-12-23T22:21:18.698968,2025-12-23T22:21:18.698968,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,1b668ad8-b3c6-4768-b747-32e1bcbdde79,2025-12-23T22:21:18.698968,{},,android
1,2627969,b7e7adb8-1c83-4feb-9047-e2e7311fcd05,c158d2f7-953d-49e8-b28d-83c97edb0fba,8274.450071,2025-12-23T22:21:18.698968,2025-12-23T22:21:18.698968,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,3d79726d-638c-4e8f-b5a0-80d820ecd41b,2025-12-23T22:21:18.698968,{},,android
2,3185278,e9d72d65-2853-4874-8ab8-fd7ed44b3755,28b1cb03-b22b-49c2-861d-e807f2fbda47,8609.106215,2025-12-23T22:21:18.698968,2025-12-23T22:21:18.698968,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,cc01fe80-2712-4c87-bddf-7525dfc795c5,2025-12-23T22:21:18.698968,{},,ios
3,2894155,a5b13a40-fbdc-4cb2-974b-ff245ee45dc1,5d454dfc-1f3c-4d85-8785-57396b9c8a82,24994.975572,2025-12-23T22:21:18.698968,2025-12-23T22:21:18.698968,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,b241d03b-ab1e-4bf0-9c77-bc5bb9a965c4,2025-12-23T22:21:18.698968,{},,android
4,2463331,6fc4f3e6-2cc6-458e-b633-1773f0b622b7,71f176c5-ece4-4c3d-88b8-f292d4f8abc8,7604.708057,2025-12-23T22:21:18.699557,2025-12-23T22:21:18.699557,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,a28ed2ff-f15b-4dd0-b047-40ff74e6985a,2025-12-23T22:21:18.699557,{},,ios


In [30]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d1d62fd2-6b3c-475f-9a34-c6ccfe356f56>

# Cash V1

## Cash

##### Alpha Cash Stack Model

###### Trench 1

In [31]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench1_applied_loans_backscored_20241001_20250930 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
-- between '2024-10-01' and '2025-02-28'
>= '2024-10-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 513132e5-4e6a-4731-8e20-f72538dd7649 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (67807, 9)


In [32]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2978333,2252cc82-9d1e-4762-9b01-0eef6f723913,21cdcaf8-b090-419f-bc17-01f7ea8f49ba,0.364313,2025-12-23T22:22:34.538766,2025-12-23T22:22:34.538766,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5555065127265288, ""apps_score...",Cash September 25 Models,1d9e3220-356c-4e04-a001-76951c8c443b,2025-12-23T22:22:34.538766,{},Trench 1,Android
1,3298424,23e46f63-6f7e-4803-8c5b-d291c43902af,9a29ec4a-d31b-42e4-8d8d-a71de134f649,0.574473,2025-12-23T22:22:34.538766,2025-12-23T22:22:34.538766,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.45442938837331637, ""apps_scor...",Cash September 25 Models,b1b3abe7-9182-47b4-bc14-6dba34d5c47a,2025-12-23T22:22:34.538766,{},Trench 1,Android
2,3126162,588a4de9-eac0-44d8-b2bb-490157fe4eed,cee18f4f-2c84-4ee4-9f66-f5801853610e,0.385729,2025-12-23T22:22:34.538766,2025-12-23T22:22:34.538766,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4561493734072989, ""apps_score...",Cash September 25 Models,524f8bb7-3d72-44a0-b560-e244a5c11920,2025-12-23T22:22:34.538766,{},Trench 1,Android
3,3575902,bbf7cbdb-71d7-40a0-bf5e-e7c800d14122,2cb4aa3d-f86b-46fc-9c88-06bea79a9265,0.534220,2025-12-23T22:22:34.538766,2025-12-23T22:22:34.538766,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5739473654545985, ""apps_score...",Cash September 25 Models,7781fc26-da23-48f8-8786-14569d5e0dba,2025-12-23T22:22:34.538766,{},Trench 1,Android
4,3232343,0452d9ef-2da7-45a1-9acf-2295117ecd64,2a085f0d-2c47-423f-bb6b-1287cb928e46,0.362354,2025-12-23T22:22:34.539783,2025-12-23T22:22:34.539783,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4617659755865677, ""apps_score...",Cash September 25 Models,fe98a5e4-f970-4a61-8a28-31e97f71749e,2025-12-23T22:22:34.539783,{},Trench 1,Android


In [33]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0ee9fff8-8e30-4f39-89a9-ac5dbb42eda9>

###### Trench 2

In [34]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench2_applied_loans_backscored_20241001_20250930 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
-- between '2024-10-01' and '2025-02-28'
>= '2024-10-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 07316c5a-fc00-46d3-a863-27836b3cfd18 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42982, 9)


In [35]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1943045,b32bcafb-52eb-4e4d-a532-053fadc7001f,a4621b96-8e72-4c83-b75c-c1afa1816a55,0.337092,2025-12-23T22:22:48.781570,2025-12-23T22:22:48.781570,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5042458888473426, ""apps_score...",Cash September 25 Models,43ae41bc-26ee-4f39-9a3e-ca6199567592,2025-12-23T22:22:48.781570,{},Trench 2,Android
1,1785773,787a493f-dce9-4cbf-9c6b-676cd4416831,3e5559e4-b8cd-4328-afbe-3f701fc49142,0.364042,2025-12-23T22:22:48.781570,2025-12-23T22:22:48.781570,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4271844194476329, ""apps_score...",Cash September 25 Models,844b493f-37b3-45f7-ac44-3805e43b4cd9,2025-12-23T22:22:48.781570,{},Trench 2,Android
2,2744019,513fb6f0-5978-455a-89e4-1c63fcb18d53,635915c0-12a7-4857-b9e7-f3bd26a41d48,0.386334,2025-12-23T22:22:48.782973,2025-12-23T22:22:48.782973,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.559138567216978, ""apps_score""...",Cash September 25 Models,2fc72bd4-9f9e-4c0b-99a9-54f3ddbf9065,2025-12-23T22:22:48.782973,{},Trench 2,Android
3,2806990,bfd57f0f-ce29-4e38-a974-d05bd6902411,dc177028-dcc6-4502-b427-0d0e4e356344,0.367072,2025-12-23T22:22:48.782973,2025-12-23T22:22:48.782973,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4494770260790322, ""apps_score...",Cash September 25 Models,a9be2395-3679-444f-b043-57b8f0cd0237,2025-12-23T22:22:48.782973,{},Trench 2,Android
4,1917464,14a5c97b-d9d8-4581-ab6c-a62aa6040297,5066cca1-159d-44a1-9b8d-f3f5e36b0dae,0.799927,2025-12-23T22:22:48.782973,2025-12-23T22:22:48.782973,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5239018230803715, ""apps_score...",Cash September 25 Models,2f77334c-ff52-4168-8317-61da16bf7c17,2025-12-23T22:22:48.782973,{},Trench 2,Android


In [36]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4fa5d003-3d17-4084-8872-7d20a28ee70f>

###### Trench 3

In [37]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench3_applied_loans_backscored_20241001_20250930 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.stack_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
-- between '2024-10-01' and '2025-02-28'
>= '2024-10-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID cafd4e6a-82ff-4a5b-be82-483a29008c2f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (18269, 9)


In [38]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1087493,0256c5fa-bc9b-471b-9653-ad1392f57ae8,b1c95d8e-b4af-431d-bb48-c319fcec0ea2,0.490953,2025-12-23T22:22:58.912752,2025-12-23T22:22:58.912752,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4770515550709822, ""apps_score...",Cash September 25 Models,c8268dde-246c-436f-bfa7-5fff9fe1ae18,2025-12-23T22:22:58.912752,{},Trench 3,Android
1,2780783,e14b7c61-d6e9-434c-8724-253c7d738844,5d9fb87e-8335-448b-a5e0-55fb45de3520,0.302832,2025-12-23T22:22:58.912752,2025-12-23T22:22:58.912752,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4425783186287803, ""apps_score...",Cash September 25 Models,25e7d348-2f19-4204-b5d6-0588bb37e829,2025-12-23T22:22:58.912752,{},Trench 3,Android
2,2506786,442d059c-d714-42b5-ad2a-9acdc14e9dbe,b77d2683-26a6-4e6c-ba5e-08cf4ca8f6f8,0.529584,2025-12-23T22:22:58.912752,2025-12-23T22:22:58.912752,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.44599151327790015, ""apps_scor...",Cash September 25 Models,96bae11a-3cde-4385-a509-8bd90e3aa12e,2025-12-23T22:22:58.912752,{},Trench 3,Android
3,2716665,4edbef81-ffca-47a7-b3a0-83b429c463a1,614ebbb6-662a-4618-aa61-bf964c53163e,0.182735,2025-12-23T22:22:58.912752,2025-12-23T22:22:58.912752,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.3993558791319496, ""apps_score...",Cash September 25 Models,ae4df143-8042-419c-aa1c-cb80dbabd223,2025-12-23T22:22:58.912752,{},Trench 3,Android
4,2224169,fa34eeda-2a67-4ee4-b1d2-5cfc3b5d51ff,04f37942-ed10-41a3-8c22-deabaa2717c8,0.571461,2025-12-23T22:22:58.913749,2025-12-23T22:22:58.913749,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5308408097845431, ""apps_score...",Cash September 25 Models,c349c579-018d-42e4-bb31-9ee26f5e04bc,2025-12-23T22:22:58.913749,{},Trench 3,Android


In [39]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6d814539-af3e-4628-95f2-44c7eda8625d>

##### Alpha Cash CIC Model

###### Trench 1

In [40]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
and r.ca_cic_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
-- between '2024-09-01' and '2025-01-31'
>= '2024-09-01'
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 89989926-d93d-48e3-a665-7cf22aa0eb12 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (77967, 17)


In [41]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'ca_cic_score',
       'max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'osType'],
      dtype='object')

In [42]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3333815,5fc0dc88-7f0f-4196-b7d6-826aa60b1597,7f3e978a-0fc0-4b31-af0b-8277ed7be51f,0.475805,2025-12-23T22:23:07.624626,2025-12-23T22:23:07.624626,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 3533.0, ""ra...",Cash September 25 Models,e0b6b1b6-3ffd-4ea0-bbcc-b9b4d338d4ba,2025-12-23T22:23:07.624626,{},Trench 1,android
1,2848870,4a9c51ba-5f01-45fb-9f7b-35c550f7d872,437437df-614f-4909-991f-a8a4e29f88d0,0.484785,2025-12-23T22:23:07.624626,2025-12-23T22:23:07.624626,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1855.0, ""ra...",Cash September 25 Models,5f3d607d-b046-4fe8-8831-c32c7051611a,2025-12-23T22:23:07.624626,{},Trench 1,android
2,3612032,c6797d2b-fd9d-4ea8-8c0d-c32ecf25d2f9,e453fa55-c969-4553-bd39-5b82c3e19697,0.390328,2025-12-23T22:23:07.624626,2025-12-23T22:23:07.624626,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 3813.0, ""ra...",Cash September 25 Models,9786ed50-39f4-4a02-8f55-a3229e4ce061,2025-12-23T22:23:07.624626,{},Trench 1,ios
3,3371694,89b8b6bd-c65a-4149-b247-04cb3bc54171,e020475e-b4b6-4b3d-a74c-c16b36f0fccf,0.515349,2025-12-23T22:23:07.624626,2025-12-23T22:23:07.624626,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2266.0, ""ra...",Cash September 25 Models,416668fc-2801-4780-8e96-85f214d162a3,2025-12-23T22:23:07.624626,{},Trench 1,android
4,3312027,2df57d0b-2cef-4d76-9e31-d454e2b50fba,8b1a4e65-175e-4b1f-8fd7-17898f6c63d3,0.454940,2025-12-23T22:23:07.624626,2025-12-23T22:23:07.624626,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1573.0, ""ra...",Cash September 25 Models,b8395a7c-161d-4a5e-84fe-31b71856d29f,2025-12-23T22:23:07.624626,{},Trench 1,android


In [43]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4213ec43-9d1c-4c95-bf59-b491e2693d2b>

###### Trench 2

In [44]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
and r.ca_cic_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
-- between '2024-09-01' and '2025-01-31'
>= '2024-09-01'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 70492990-95aa-4ebf-beb7-3f17be7ee4f4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (48479, 17)


In [45]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2405862,348244c6-2460-42d4-9b9f-a2cc458881fc,d59d22a1-cb33-457b-8a84-f82248553b28,0.569542,2025-12-23T22:23:26.488754,2025-12-23T22:23:26.488754,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 5061.0, ""ra...",Cash September 25 Models,ecd5811c-7a7b-48ee-ad8e-87c252f2bdc5,2025-12-23T22:23:26.488754,{},Trench 2,ios
1,1277118,4131b592-ce30-4bf9-b9c1-6c7d30eeb3b8,30a4eb7a-6f63-4aa8-b91c-6e8c656f37b8,0.444077,2025-12-23T22:23:26.488754,2025-12-23T22:23:26.488754,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2850.0, ""ra...",Cash September 25 Models,044fa19c-f50d-4698-a00e-9464763471cf,2025-12-23T22:23:26.488754,{},Trench 2,android
2,1478187,b5bcb088-e715-4cdf-92fd-c5d82d5842d6,362ecf68-3e42-4ba7-862f-d71a05aa264c,0.563232,2025-12-23T22:23:26.488754,2025-12-23T22:23:26.488754,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1735.0, ""ra...",Cash September 25 Models,5a0943cd-8267-4cb7-b3ea-5f51c7dd08c1,2025-12-23T22:23:26.488754,{},Trench 2,ios
3,3006207,f661f250-e0e1-431c-8e03-cd0c8127d2fd,415cb733-bf04-4402-bc20-b7d938b6031d,0.739106,2025-12-23T22:23:26.489752,2025-12-23T22:23:26.489752,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1025.0, ""ra...",Cash September 25 Models,69ed5a57-a3c7-4578-bbc4-714ffbe1c1de,2025-12-23T22:23:26.489752,{},Trench 2,ios
4,3062546,505f4e42-d22b-46f8-86f0-84d1908e3706,af3e4898-657a-4ddf-ba99-cd728eacaab0,0.364949,2025-12-23T22:23:26.489752,2025-12-23T22:23:26.489752,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 5062.0, ""ra...",Cash September 25 Models,4760d7ea-fe60-47d1-bbc6-a3daed6bfeff,2025-12-23T22:23:26.489752,{},Trench 2,android


In [46]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=13515b88-5c4e-4cb0-aa38-18ea5485fe07>

###### Trench 3

In [47]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
and r.ca_cic_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
-- between '2024-09-01' and '2025-01-31'
>= '2024-09-01'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 372a20d6-956e-46cd-8b15-d99dadaf5a59 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (19300, 17)


In [48]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2121377,56c825e9-e6e1-4c16-aac9-c62f4a9ebcfb,393a3d2f-72da-437e-b7b1-1992100f0d48,0.553604,2025-12-23T22:23:40.424046,2025-12-23T22:23:40.424046,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 479.0, ""rat...",Cash September 25 Models,fff7179a-afb1-4e4b-923a-2b7306393f88,2025-12-23T22:23:40.424046,{},Trench 3,android
1,2213115,71d2eb3f-51af-4bf8-bce7-d22973d699e8,5d9ab096-bffc-4f42-8835-4fe5eeec10f6,0.438062,2025-12-23T22:23:40.425047,2025-12-23T22:23:40.425047,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1263.0, ""ra...",Cash September 25 Models,0f5e6434-c872-4848-8e69-af0fa846e6d2,2025-12-23T22:23:40.425047,{},Trench 3,ios
2,2242045,909a9973-57b5-4ecf-9369-3827548a026c,fe13bc36-8987-471d-958c-7acc67d29b9b,0.456504,2025-12-23T22:23:40.425047,2025-12-23T22:23:40.425047,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 927.0, ""rat...",Cash September 25 Models,fd1e1814-bbd7-4e1a-a755-ede0e567c2ce,2025-12-23T22:23:40.425047,{},Trench 3,android
3,3028821,a701f2ca-3a48-4bd0-b550-7b19e9586ae5,ec9c194e-926d-4d9e-9965-7004dac0a302,0.397166,2025-12-23T22:23:40.425047,2025-12-23T22:23:40.425047,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2738.0, ""ra...",Cash September 25 Models,e0ccffa4-0dbb-4e83-a57a-461556a904c3,2025-12-23T22:23:40.425047,{},Trench 3,android
4,2515943,a0cfbdee-614c-414e-b2a9-aee5433dd2fc,cf0ca920-c0a1-42e3-9280-e0f48255960f,0.597315,2025-12-23T22:23:40.425047,2025-12-23T22:23:40.425047,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 336.0, ""rat...",Cash September 25 Models,f6bbfd64-9951-4355-a8fb-9a3220cf392c,2025-12-23T22:23:40.425047,{},Trench 3,android


In [49]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c4165727-17e7-48c6-8e46-3bc64454ffb5>

##### Beta-Cash-Demo-Model

###### Trench 1

In [50]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
and r.c_demo_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
-- between '2024-10-01' and '2025-01-31'
>= '2024-10-01'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 54f27ba9-aab7-4799-a052-b1d7aaea5222 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (324441, 16)


In [51]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3633538,2d53bc0f-8d23-4bf4-bdb7-11d76f2ce86b,f062fd5c-3592-440f-90e5-5f31f2c6c109,0.548851,2025-12-23T22:23:53.476515,2025-12-23T22:23:53.476515,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,25fd3b7d-7a01-4505-8b99-2eaf507fd70d,2025-12-23T22:23:53.476515,{},Trench 1,ios
1,3619053,d1c2505d-50e8-40cb-b3ad-380344cf536c,8651d033-93f8-48a2-8b47-43ce1a4d8668,0.336607,2025-12-23T22:23:53.477517,2025-12-23T22:23:53.477517,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,efcbe486-d687-4863-97cb-f92fe1b4b3c0,2025-12-23T22:23:53.477517,{},Trench 1,ios
2,3022091,481396a6-4482-41a4-8147-7fe7414018e1,cf2a60a5-109b-4b57-848b-f12899287b31,0.581995,2025-12-23T22:23:53.477517,2025-12-23T22:23:53.477517,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,33aeb46d-c20e-4a02-93f6-b776b0493e05,2025-12-23T22:23:53.477517,{},Trench 1,android
3,3215692,d582f939-fcfd-4530-afcf-422c0970aea4,b1bda79b-3857-42b4-956e-2470382eff45,0.337885,2025-12-23T22:23:53.478517,2025-12-23T22:23:53.478517,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,657a3744-6a88-40fd-9204-7b4d8fe4caff,2025-12-23T22:23:53.478517,{},Trench 1,ios
4,3646085,4ec7cc47-49c2-469c-8a08-cbb980f6f147,99e3d8f9-a3b7-4f94-80d9-7c0e088d516a,0.550242,2025-12-23T22:23:53.478517,2025-12-23T22:23:53.478517,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,5b91063a-f4ac-4b25-823c-5ccc50d148cf,2025-12-23T22:23:53.478517,{},Trench 1,ios


In [52]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=02cfedd4-d7d7-41c5-b4ae-b3aeae135b65>

###### Trench 2

In [53]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
and r.c_demo_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
-- between '2024-10-01' and '2025-10-31'
>= '2024-10-01'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID b98f76fc-c6dd-426d-9f43-c03343f098c0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (124539, 16)


In [54]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2965438,5bff9433-8ed9-4551-995b-e3c2348f5eea,52fa7cdd-729a-4e59-a307-74433d65af7f,0.342913,2025-12-23T22:24:55.298479,2025-12-23T22:24:55.298479,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,73c3d402-45b0-4331-9dfc-a4e4742dd649,2025-12-23T22:24:55.298479,{},Trench 2,android
1,1995137,b31e0936-7aa1-41df-811e-c61d773c1dcb,b8a47d43-b4e2-415c-9016-781ca3893ad7,0.362515,2025-12-23T22:24:55.298479,2025-12-23T22:24:55.298479,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,07f88b0c-375b-41e6-9ced-c13c6b766141,2025-12-23T22:24:55.298479,{},Trench 2,android
2,3180216,abd7d094-6dc4-41ae-821b-a2d72375b174,b30f4e03-770f-48ba-b1f2-933310247b3c,0.491307,2025-12-23T22:24:55.298479,2025-12-23T22:24:55.298479,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,5c08017a-02b0-4e15-924e-e3ab1630d064,2025-12-23T22:24:55.298479,{},Trench 2,android
3,3068448,5455ca4a-df8d-4cd2-9f79-1eb80346da0c,c7ced19a-5c21-4985-baa5-d466ecb48948,0.525217,2025-12-23T22:24:55.298479,2025-12-23T22:24:55.298479,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,a2af96cd-e8c1-4fb5-ae0a-8dafd1f9747e,2025-12-23T22:24:55.298479,{},Trench 2,android
4,2983185,643f2d11-b3a1-45cf-bb23-41ac4b768da7,298458d3-207f-49fb-aa87-0bbe4605c1f9,0.378537,2025-12-23T22:24:55.299475,2025-12-23T22:24:55.299475,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,3ca52c3e-2da8-4ce8-a5df-1e641f422da6,2025-12-23T22:24:55.299475,{},Trench 2,ios


In [55]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=604f0669-f985-4a58-aff5-1959cd7b09cc>

###### Trench 3

In [56]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
and r.c_demo_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
-- between '2024-10-01' and '2025-01-31'
>= '2024-10-01'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 0ef729fe-27da-42fe-9cdc-b08eb53c845e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42441, 16)


In [57]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1663369,ad9418b7-64b4-4a86-83b2-73be56e73b2c,64939ecf-1e68-4321-ac7d-c836fc4b256c,0.302997,2025-12-23T22:25:22.742908,2025-12-23T22:25:22.742908,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,fb369546-9958-4666-860b-7e046c422419,2025-12-23T22:25:22.742908,{},Trench 3,android
1,3011579,5be134fd-d61d-46ab-aeb8-78a99806d222,65a5653b-8343-42f8-973b-4a2b69f0e838,0.408293,2025-12-23T22:25:22.742908,2025-12-23T22:25:22.742908,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,b8e5fd88-b3c8-4245-9c42-1a034267fbf8,2025-12-23T22:25:22.742908,{},Trench 3,android
2,2870649,0db2aac5-87b8-4906-9257-4c3c3695c578,f4afeb3a-f5ac-48ed-b49b-e4c7fe8e6f31,0.476591,2025-12-23T22:25:22.742908,2025-12-23T22:25:22.742908,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,ed29b60b-b42b-4bcb-b18b-68dcca6dc18b,2025-12-23T22:25:22.742908,{},Trench 3,android
3,2851586,050497d4-80d3-4965-9ebb-e562758719e0,c954897a-0f64-487f-9524-f6f7f4136a9b,0.331474,2025-12-23T22:25:22.742908,2025-12-23T22:25:22.742908,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,d9467efe-9807-4b87-9534-1377b719b3b7,2025-12-23T22:25:22.742908,{},Trench 3,android
4,2541413,c1804b19-71da-44d6-9a60-c0d2ee57ce57,de591019-f8a8-4abd-8048-4738e53ae50d,0.610189,2025-12-23T22:25:22.742908,2025-12-23T22:25:22.742908,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,f38bacd2-50f2-4142-ac32-c90d2748d7f6,2025-12-23T22:25:22.742908,{},Trench 3,android


In [58]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=af27ac29-b38f-4fc2-b644-25a07eaf55f2>

##### Beta-Cash-Stack-Model

In [59]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench1_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
 where r.stack_score is not null 
 and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
-- between '2024-10-01' and '2025-02-28'
>= '2024-10-01'
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 5e27473b-5acc-4f68-9a53-f18a03ca669e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (324344, 8)


In [60]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3628812,9becea11-0967-41b5-b3bb-899658ce6fff,cd99832c-25f7-4b57-94c8-855e502ded85,0.393886,2025-12-23T22:25:39.539602,2025-12-23T22:25:39.539602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.6432102610853918, ""apps_score...",Cash September 25 Models,5f156dad-03d8-4bb2-aa65-1e099203a948,2025-12-23T22:25:39.539602,{},Trench 1,android
1,3570757,03a33509-8eee-4015-aa3e-7cabda480862,0efe52a5-27f7-4c72-83f4-0757733ce2e6,0.629153,2025-12-23T22:25:39.539602,2025-12-23T22:25:39.539602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5180475486489378, ""apps_score...",Cash September 25 Models,72c158a9-cb80-48aa-8d31-bbc45314021a,2025-12-23T22:25:39.539602,{},Trench 1,android
2,3076277,f73e2d76-67e1-4c55-9d44-15094ac97f27,c8114e52-ef3c-45f7-8031-bb7e5cdf82d9,0.947176,2025-12-23T22:25:39.539602,2025-12-23T22:25:39.539602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.45492391822126593, ""apps_scor...",Cash September 25 Models,fc9df23b-386f-458e-8399-7ced22a60e25,2025-12-23T22:25:39.539602,{},Trench 1,android
3,2949134,96f0d55e-6c24-421f-b6d7-d1cdfe9f0e82,4a959386-9a05-4999-b358-45f832ee9883,0.310804,2025-12-23T22:25:39.540602,2025-12-23T22:25:39.540602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4331471291485455, ""apps_score...",Cash September 25 Models,3d6ff42c-f3dd-4ae0-9531-7611eb3558d1,2025-12-23T22:25:39.540602,{},Trench 1,android
4,3105318,71e26e23-e057-45fd-bda7-8f52607d13e3,64624604-720b-41b6-addb-797fb3838b85,0.745564,2025-12-23T22:25:39.540602,2025-12-23T22:25:39.540602,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5518455179500804, ""apps_score...",Cash September 25 Models,b9cf4fc0-2fbe-4657-8964-b3f9e30aabe7,2025-12-23T22:25:39.540602,{},Trench 1,android


In [61]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ff7ed17a-1eb2-4e30-8407-084372123cc5>

###### Trench 2

In [62]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench2_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
 where r.stack_score is not null
 and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
-- between '2024-10-01' and '2025-02-28'
>= '2024-10-01'
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID ec2977b9-02ea-4328-bc4a-46fb70bedfa4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (121682, 8)


In [63]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2805551,df01fa2f-dd21-4856-ab71-11a14db98c05,aad4fb20-acff-44ae-bb9f-fec9fed8f362,0.429548,2025-12-23T22:26:21.795762,2025-12-23T22:26:21.795762,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5996353702617035, ""apps_score...",Cash September 25 Models,cdd9c588-9da8-410c-a37c-79fe8a234f52,2025-12-23T22:26:21.795762,{},Trench 2,android
1,3386815,d3b69c02-b19b-4513-8b78-e77adf001a81,0cf77b49-bf72-4f2f-a275-1714b5fa9dd5,0.564492,2025-12-23T22:26:21.795762,2025-12-23T22:26:21.795762,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.6009622553308236, ""apps_score...",Cash September 25 Models,35ecb666-ed9c-4cb0-9ba3-722556e9a3af,2025-12-23T22:26:21.795762,{},Trench 2,android
2,1996190,6348e080-20be-489a-9cb6-7101cd83184d,4ffc31f8-820f-4b6c-9e37-a2f16e5fd2ec,0.258253,2025-12-23T22:26:21.795762,2025-12-23T22:26:21.795762,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.48757588969484456, ""apps_scor...",Cash September 25 Models,f8fb831c-ef9f-487c-bd1f-0b00d00917a4,2025-12-23T22:26:21.795762,{},Trench 2,android
3,3073927,29a8c6db-768d-4e52-960a-034a3a8469d6,e7d93035-17d4-407c-afc8-2c2ddf7bb230,0.394901,2025-12-23T22:26:21.795762,2025-12-23T22:26:21.795762,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.34027830850667656, ""apps_scor...",Cash September 25 Models,d11e60dd-186d-44da-a376-c121aa981cb5,2025-12-23T22:26:21.795762,{},Trench 2,android
4,1319586,8248f3c7-529e-423c-ad5a-fe2604146690,4adc93a4-0c34-4761-952c-96ab9d7e1220,0.150396,2025-12-23T22:26:21.795762,2025-12-23T22:26:21.795762,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.49227695930346327, ""apps_scor...",Cash September 25 Models,38bb2bfc-4f8c-4771-99f2-6b99597de177,2025-12-23T22:26:21.795762,{},Trench 2,android


In [64]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=92bcae6f-2b81-453c-8aa2-4d7b19a4f722>

###### Trench 3

In [65]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench3_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
 where r.stack_score is not null 
 and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-10-01' and '2025-02-28'
 >= '2024-10-01'

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 3b5f9d62-5baa-4d4a-9a1d-c733cfce3c2e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42414, 8)


In [66]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2171825,00069ad5-f245-4299-a021-3d2edade1c77,d6288a52-f631-412b-962e-326cbf0ccfa9,0.646851,2025-12-23T22:26:42.914855,2025-12-23T22:26:42.914855,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.3803655850699276, ""apps_score...",Cash September 25 Models,cf4476b9-edd4-4d60-9b2f-13aed3698a2f,2025-12-23T22:26:42.914855,{},Trench 3,android
1,2844386,05f16b70-16bd-4879-83e4-3467ead3ff10,47043c3f-d727-4764-9d30-5bb7abcc7abd,0.587956,2025-12-23T22:26:42.914855,2025-12-23T22:26:42.914855,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.47127665050833095, ""apps_scor...",Cash September 25 Models,96e277ea-6d7d-41c5-8dd4-17c4a533dd9b,2025-12-23T22:26:42.914855,{},Trench 3,android
2,3245163,83a5501f-e2b9-44a6-b492-07ac54e483a8,a84bd705-961f-4f30-9c8b-c1f142f94a10,0.800269,2025-12-23T22:26:42.914855,2025-12-23T22:26:42.914855,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5025959195767765, ""apps_score...",Cash September 25 Models,22f589ad-3c2c-4b32-94c1-dc0e3c3a644e,2025-12-23T22:26:42.914855,{},Trench 3,android
3,3243835,95416304-b4fa-4f59-a94a-5c4425d5ebe2,9350c876-eaf9-453c-b1bd-9604677fa8a2,0.689667,2025-12-23T22:26:42.914855,2025-12-23T22:26:42.914855,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5383533435645543, ""apps_score...",Cash September 25 Models,c33d260a-ab55-4905-9458-c152a0370065,2025-12-23T22:26:42.914855,{},Trench 3,android
4,3232459,d0913f75-aefa-4274-a679-371f65a8ab37,3e1f850b-1e24-4fe2-be9e-a5a508fa4ef1,0.642559,2025-12-23T22:26:42.914855,2025-12-23T22:26:42.914855,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.47332194079062734, ""apps_scor...",Cash September 25 Models,06853918-9b28-4027-baab-e61b983747b1,2025-12-23T22:26:42.914855,{},Trench 3,android


In [67]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f0dbdaa4-e56e-49e1-87a8-eb8c4616c3a4>

#####  Beta Cash AppScore Model

###### Trench 1

In [68]:
## query to add the trench category trench 1 and trench 2

# create or replace table risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy as 
# select * from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored;



# ALTER TABLE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy
# ADD COLUMN trench_category STRING;

# MERGE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy AS target
# USING (
#   SELECT 
#     a.digitalLoanAccountId,
#     CASE 
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) <= 30 THEN 'Trench 1'
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) > 30 THEN 'Trench 2'
#       WHEN b.ln_loan_level_user_type = '1_Repeat Applicant' THEN 'Trench 3'
#       ELSE NULL
#     END AS trench_category,
#     ROW_NUMBER() OVER (PARTITION BY a.digitalLoanAccountId ORDER BY b.ln_appln_submit_datetime DESC) AS row_num
#   FROM risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy a
#   LEFT JOIN prj-prod-dataplatform.risk_mart.applied_loans_20210701_20250930_trans b
#     ON a.digitalLoanAccountId = b.digitalLoanAccountId
# ) AS source
# ON target.digitalLoanAccountId = source.digitalLoanAccountId
# WHEN MATCHED AND source.row_num = 1 THEN
#   UPDATE SET target.trench_category = source.trench_category;

In [69]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and r.apps_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-08-13' and '2025-01-31'
 >= '2024-08-13'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID bd15a437-58cc-4206-9eec-dcbe842c2641 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (271448, 12)


In [70]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d', 'osType'],
      dtype='object')

In [71]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271448 entries, 0 to 271447
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            271448 non-null  int64  
 1   digitalLoanAccountId  271448 non-null  object 
 2   crifApplicationId     271448 non-null  object 
 3   prediction            271448 non-null  float64
 4   start_time            271448 non-null  object 
 5   end_time              271448 non-null  object 
 6   modelDisplayName      271448 non-null  object 
 7   modelVersionId        271448 non-null  object 
 8   calcFeature           271448 non-null  object 
 9   subscription_name     271448 non-null  object 
 10  message_id            271448 non-null  object 
 11  publish_time          271448 non-null  object 
 12  attributes            271448 non-null  object 
 13  trenchCategory        271448 non-null  object 
 14  deviceOs              271448 non-null  object 
dtype

In [72]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9e210f25-4d31-4268-b462-893c1ac88a29>

###### Trench 2

In [73]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and r.apps_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-08-13' and '2025-01-31'
 >= '2024-08-13'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 71cfeb81-fa65-40af-a152-709da96d051c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (92944, 12)


In [74]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92944 entries, 0 to 92943
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            92944 non-null  int64  
 1   digitalLoanAccountId  92944 non-null  object 
 2   crifApplicationId     92944 non-null  object 
 3   prediction            92944 non-null  float64
 4   start_time            92944 non-null  object 
 5   end_time              92944 non-null  object 
 6   modelDisplayName      92944 non-null  object 
 7   modelVersionId        92944 non-null  object 
 8   calcFeature           92944 non-null  object 
 9   subscription_name     92944 non-null  object 
 10  message_id            92944 non-null  object 
 11  publish_time          92944 non-null  object 
 12  attributes            92944 non-null  object 
 13  trenchCategory        92944 non-null  object 
 14  deviceOs              92944 non-null  object 
dtypes: float64(1), int6

In [75]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=077d0499-7e79-4812-b9f6-c8668b3f498d>

###### Trench 3

In [76]:
sq = """   
 select 
  r.customerId customer_id, 
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,
  r.app_cnt_health_and_fitness_ever_binned app_cnt_health_and_fitness_ever,
  r.app_cnt_productivity_ever_binned app_cnt_productivity_ever, 
  r.app_cnt_rated_for_18plus_ever_binned app_cnt_rated_for_18plus_ever,
  r.app_cnt_books_and_reference_ever_binned app_cnt_books_and_reference_ever, 
  r.app_cnt_gaming_180d_binned app_cnt_gaming_180d,
  r.app_cnt_absence_tag_365d_binned app_cnt_absence_tag_365d,
  r.app_last_payday_install_to_apply_days_binned app_last_payday_install_to_apply_days,
  r.app_cnt_absence_tag_365d_binned,
  r.app_cnt_gaming_180d_binned,
  r.app_cnt_productivity_ever_binned,
  r.app_cnt_rated_for_18plus_ever_binned,
  r.app_cnt_health_and_fitness_ever_binned,
  r.app_cnt_books_and_reference_ever_binned,
  r.app_last_payday_install_to_apply_days_binned,
  r.ln_user_type,
  case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.b_score_model_applied_loans_cash_20240101_20250930_app_scored r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.apps_score is not null 
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-08-13' and '2025-01-31'
 >= '2024-08-13'

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID dd028057-025f-4eaf-a486-24fe39b733ee successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (57212, 19)


In [77]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_productivity_ever',
       'app_cnt_rated_for_18plus_ever', 'app_cnt_books_and_reference_ever',
       'app_cnt_gaming_180d', 'app_cnt_absence_tag_365d',
       'app_last_payday_install_to_apply_days',
       'app_cnt_absence_tag_365d_binned', 'app_cnt_gaming_180d_binned',
       'app_cnt_productivity_ever_binned',
       'app_cnt_rated_for_18plus_ever_binned',
       'app_cnt_health_and_fitness_ever_binned',
       'app_cnt_books_and_reference_ever_binned',
       'app_last_payday_install_to_apply_days_binned', 'ln_user_type']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1547909,fad0f179-310f-465b-9f73-302e61837ff3,ceb03fe2-7e49-493e-a557-0487f83f021b,0.485871,2025-12-23T22:28:05.204362,2025-12-23T22:28:05.204362,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.48587059914585395, ""...",Cash September 25 Models,d63d67e3-c9db-48e2-bc19-e17a3dc387dc,2025-12-23T22:28:05.204362,{},Trench 3,android
1,2259608,d1cf1838-3e2a-4f47-b8f7-5a21c0d51690,e861dec8-4555-4b2a-a675-054208503491,0.550633,2025-12-23T22:28:05.204362,2025-12-23T22:28:05.204362,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5506327720064113, ""a...",Cash September 25 Models,956c6b59-59c2-45b2-bf5b-ff7c3368a467,2025-12-23T22:28:05.204362,{},Trench 3,android
2,3257260,3ed1fd5d-047d-4774-b0c7-465fdce70303,fc2f98ec-49fd-4ccb-a9c8-8fd6f6bb2a6b,0.552340,2025-12-23T22:28:05.204362,2025-12-23T22:28:05.204362,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5523403109687754, ""a...",Cash September 25 Models,32a7c0ee-cc42-4ec4-9057-f20987358b66,2025-12-23T22:28:05.204362,{},Trench 3,android
3,2415785,3056df06-8e64-4b8f-8b31-159341e29b83,28b50e28-a73f-4345-905b-d94d27ccc4c8,0.475436,2025-12-23T22:28:05.204362,2025-12-23T22:28:05.204362,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.47543639113910013, ""...",Cash September 25 Models,21f9f6d7-dbbc-4b3e-b82a-5e7c2c7ab162,2025-12-23T22:28:05.204362,{},Trench 3,android
4,2427025,808a15cf-e79f-475d-8066-9a09fd16da4d,b03c57ad-178c-4946-b799-5e7c4d88fc02,0.527214,2025-12-23T22:28:05.204362,2025-12-23T22:28:05.204362,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5272138908109254, ""a...",Cash September 25 Models,d68560ce-23aa-4f35-b562-5cfed065c34d,2025-12-23T22:28:05.204362,{},Trench 3,android


In [78]:
dfd.info()
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57212 entries, 0 to 57211
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            57212 non-null  object 
 1   digitalLoanAccountId  57212 non-null  object 
 2   crifApplicationId     57212 non-null  object 
 3   prediction            57212 non-null  float64
 4   start_time            57212 non-null  object 
 5   end_time              57212 non-null  object 
 6   modelDisplayName      57212 non-null  object 
 7   modelVersionId        57212 non-null  object 
 8   calcFeature           57212 non-null  object 
 9   subscription_name     57212 non-null  object 
 10  message_id            57212 non-null  object 
 11  publish_time          57212 non-null  object 
 12  attributes            57212 non-null  object 
 13  trenchCategory        57212 non-null  object 
 14  deviceOs              57212 non-null  object 
dtypes: float64(1), obje

In [79]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=70954453-e614-49e5-a702-09b2736ad792>

# SIL V2

*   Sil-Alpha-CIC-SIL-Model: worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930
*   Sil-Alpha-StackingModel: worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3
*   Sil-Beta-AppsScoreModel:
*   New Applicants (T1, T2): risk_mart.applied_sil_trench1_trench2_loan_jan2024_30sep2025_app_scored, risk_mart.applied_sil_new_applicants_loan_oct0125_oct1325_app_scored
*   Repeat Applicants (T3): risk_mart.b_score_model_applied_loans_sil_20240101_20250930_app_scored, risk_mart.applied_sil_repeat_applicants_loan_oct0125_oct1325_app_scored
*   Sil-Beta-DemoScoreModel: worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015
*   Sil-Beta-StackScoreModel: worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3m

##### 'Alpha - CIC-SIL-Model'

##### Trench 1

In [80]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId, 
r.c_cic_score ,
r.ScoreRange,
       ln_loan_level_user_type, flg_zero_non_granted_ever,
       flg_zero_granted_ever,
       Personal_Loans_granted_contracts_amt_24M,
       granted_contracts_cnt_6M, total_overdue_granted_contracts,
       has_ever_been_overdue, cnt_nongranted_contracts_3M,
       cnt_active_contracts, max_amt_granted_24M,
       tot_active_contracts_util, days_since_last_closed,
       vel_contract_nongranted_cnt_6on12,
       vel_contract_granted_amt_6on12,
       vel_contract_closed_amt_3on12,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType
from worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and r.c_cic_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-09-01' and '2025-01-31'
 >= '2024-09-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 81fff2a0-7c64-4dc0-94c2-8026a7345071 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (224718, 20)


In [81]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'c_cic_score', 'ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12', 'osType'],
      dtype='object')

In [82]:
data.head()

,customer_id,digitalLoanAccountId,c_cic_score,ScoreRange,ln_loan_level_user_type,flg_zero_non_granted_ever,flg_zero_granted_ever,Personal_Loans_granted_contracts_amt_24M,granted_contracts_cnt_6M,total_overdue_granted_contracts,has_ever_been_overdue,cnt_nongranted_contracts_3M,cnt_active_contracts,max_amt_granted_24M,tot_active_contracts_util,days_since_last_closed,vel_contract_nongranted_cnt_6on12,vel_contract_granted_amt_6on12,vel_contract_closed_amt_3on12,osType
0,3403096,6ff38e35-c575-4dbd-90f0-0aa61c6edc84,0.289316,Ci,2_New Applicant,0,0,13857.0,5.0,10.0,1.0,NaN,2.0,3700.0,1.140726,21.0,2.012048,0.792661,1.553707,android
1,3318115,d66122fc-928c-419d-b75a-d1a478f855a4,0.298109,Missing,2_New Applicant,1,0,65092.0,NaN,7.0,1.0,NaN,2.0,37389.0,0.000000,101.0,NaN,NaN,NaN,ios
2,3654928,6d529da6-28e9-4daf-8451-2eca1e8ae6eb,0.335536,Gi,2_New Applicant,0,0,30708.0,1.0,22.0,1.0,NaN,1.0,25902.0,1.000000,452.0,NaN,1.000000,NaN,android
3,3652577,ccc3514e-4508-46d6-9acb-f384d26b2073,0.215495,Missing,2_New Applicant,0,0,108239.0,NaN,10.0,1.0,NaN,3.0,28844.0,0.527881,2.0,2.012048,NaN,4.410464,android
4,3674326,913fa91f-f4ea-48fa-b654-844e4fca17b8,0.421369,Bi,2_New Applicant,0,0,373.0,NaN,5.0,1.0,1.0,NaN,212.0,NaN,216.0,2.012048,NaN,NaN,android


In [83]:
feature_column = ['ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12',]

dfd = transform_datav2(data, feature_column, a='c_cic_score', modelDisplayName='cic_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3403096,6ff38e35-c575-4dbd-90f0-0aa61c6edc84,9dd016b6-dcc7-413f-9aab-40edabf35745,0.289316,2025-12-23T22:28:24.229940,2025-12-23T22:28:24.229940,cic_model_sil,v2,"{""ScoreRange"": ""Ci"", ""ln_loan_level_user_type""...",Cash November 25 Models,f39967ff-b454-4659-a43f-013ecd8b416a,2025-12-23T22:28:24.229940,{},Trench 1,android
1,3318115,d66122fc-928c-419d-b75a-d1a478f855a4,565ee102-2bc3-4291-9bad-2468587b80d3,0.298109,2025-12-23T22:28:24.229940,2025-12-23T22:28:24.229940,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,7b4641fe-ad6f-478f-87e5-f408c1fefe4f,2025-12-23T22:28:24.229940,{},Trench 1,ios
2,3654928,6d529da6-28e9-4daf-8451-2eca1e8ae6eb,2377f69e-6502-4906-a349-04ccc3f61d86,0.335536,2025-12-23T22:28:24.230459,2025-12-23T22:28:24.230459,cic_model_sil,v2,"{""ScoreRange"": ""Gi"", ""ln_loan_level_user_type""...",Cash November 25 Models,c69b350d-5b79-4132-aa8a-1c240296683b,2025-12-23T22:28:24.230459,{},Trench 1,android
3,3652577,ccc3514e-4508-46d6-9acb-f384d26b2073,c11eb49a-6c27-448a-a4ec-d6a60773c2e1,0.215495,2025-12-23T22:28:24.230459,2025-12-23T22:28:24.230459,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,b2d0df2f-99af-4a68-9a18-dbccbaeeaddc,2025-12-23T22:28:24.230459,{},Trench 1,android
4,3674326,913fa91f-f4ea-48fa-b654-844e4fca17b8,e6de1c3b-fae1-4897-967e-d8e180af9c69,0.421369,2025-12-23T22:28:24.230459,2025-12-23T22:28:24.230459,cic_model_sil,v2,"{""ScoreRange"": ""Bi"", ""ln_loan_level_user_type""...",Cash November 25 Models,ade84529-afea-4e59-b6e9-dc134678f657,2025-12-23T22:28:24.230459,{},Trench 1,android


In [84]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=255ea07a-ef20-4b2e-a049-1228862e84b8>

##### Trench 2

In [85]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId, 
r.c_cic_score ,
r.ScoreRange,
       ln_loan_level_user_type, flg_zero_non_granted_ever,
       flg_zero_granted_ever,
       Personal_Loans_granted_contracts_amt_24M,
       granted_contracts_cnt_6M, total_overdue_granted_contracts,
       has_ever_been_overdue, cnt_nongranted_contracts_3M,
       cnt_active_contracts, max_amt_granted_24M,
       tot_active_contracts_util, days_since_last_closed,
       vel_contract_nongranted_cnt_6on12,
       vel_contract_granted_amt_6on12,
       vel_contract_closed_amt_3on12,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType
from worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and r.c_cic_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-09-01' and '2025-01-31'
 >= '2024-09-01'
 ;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 468bb814-96cb-432c-99b1-36d5f4fc49ad successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (10175, 20)


In [86]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'c_cic_score', 'ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12', 'osType'],
      dtype='object')

In [87]:
data.head()

,customer_id,digitalLoanAccountId,c_cic_score,ScoreRange,ln_loan_level_user_type,flg_zero_non_granted_ever,flg_zero_granted_ever,Personal_Loans_granted_contracts_amt_24M,granted_contracts_cnt_6M,total_overdue_granted_contracts,has_ever_been_overdue,cnt_nongranted_contracts_3M,cnt_active_contracts,max_amt_granted_24M,tot_active_contracts_util,days_since_last_closed,vel_contract_nongranted_cnt_6on12,vel_contract_granted_amt_6on12,vel_contract_closed_amt_3on12,osType
0,2841718,8f1387e2-340b-4337-9ec9-e3016052f87c,0.544667,Missing,2_New Applicant,0,0,10000.0,1.0,12.0,1.0,1.0,NaN,8000.0,NaN,155.0,1.000,1.000000,NaN,android
1,2673297,59ee5e5a-b93b-446d-963e-c0f107bcb3bd,0.331334,Missing,2_New Applicant,0,0,38092.0,2.0,22.0,1.0,NaN,2.0,35050.0,0.972801,65.0,1.000,6.318314,7.810791,android
2,3506911,9128cd04-a07f-43dc-bf52-1e7b117d8fc4,0.244780,NH_Ii,2_New Applicant,1,0,NaN,NaN,9.0,1.0,NaN,5.0,NaN,0.923462,1827.0,NaN,NaN,NaN,android
3,2495360,c7875f9b-97ec-4ba3-afee-f62a048547c6,0.323431,Missing,2_New Applicant,0,0,30795.0,NaN,9.0,1.0,1.0,11.0,20000.0,1.498466,181.0,0.668,NaN,NaN,android
4,1819674,751ecf1d-a258-4da0-a1fd-47156c1a535b,0.374450,Ei,2_New Applicant,0,0,226396.0,1.0,10.0,1.0,2.0,6.0,358000.0,0.903271,72.0,1.332,1.242231,1.477571,android


In [88]:
feature_column = ['ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12',]

dfd = transform_datav2(data, feature_column, a='c_cic_score', modelDisplayName='cic_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2841718,8f1387e2-340b-4337-9ec9-e3016052f87c,5fbcefa6-3117-40a1-9528-b386363f0962,0.544667,2025-12-23T22:29:09.545118,2025-12-23T22:29:09.545118,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,113bae29-5670-4167-b814-6228fcc8f451,2025-12-23T22:29:09.545118,{},Trench 2,android
1,2673297,59ee5e5a-b93b-446d-963e-c0f107bcb3bd,5abb2b6e-8072-4cd4-84bf-c68af8e7bbde,0.331334,2025-12-23T22:29:09.546120,2025-12-23T22:29:09.546120,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,44ebb3cf-634a-428e-8c53-96040304710a,2025-12-23T22:29:09.546120,{},Trench 2,android
2,3506911,9128cd04-a07f-43dc-bf52-1e7b117d8fc4,d467988e-879f-4537-b743-4906179a0b26,0.244780,2025-12-23T22:29:09.546120,2025-12-23T22:29:09.546120,cic_model_sil,v2,"{""ScoreRange"": ""NH_Ii"", ""ln_loan_level_user_ty...",Cash November 25 Models,941b7ed9-ada0-415b-b98b-9303bde6471b,2025-12-23T22:29:09.546120,{},Trench 2,android
3,2495360,c7875f9b-97ec-4ba3-afee-f62a048547c6,bf51d0b5-c895-44f6-bb4a-1b032ea64a9e,0.323431,2025-12-23T22:29:09.546120,2025-12-23T22:29:09.546120,cic_model_sil,v2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Cash November 25 Models,01bd1fc8-3966-471d-806b-c9cce6b8cc4d,2025-12-23T22:29:09.546120,{},Trench 2,android
4,1819674,751ecf1d-a258-4da0-a1fd-47156c1a535b,1ca0daf4-693b-4608-b150-50ddae3796c2,0.374450,2025-12-23T22:29:09.546120,2025-12-23T22:29:09.546120,cic_model_sil,v2,"{""ScoreRange"": ""Ei"", ""ln_loan_level_user_type""...",Cash November 25 Models,02b05304-2ed1-4775-9558-32fccac6ce62,2025-12-23T22:29:09.546120,{},Trench 2,android


In [89]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ed44ec55-6d07-4697-9b3a-c8a209835836>

##### Trench 3

In [90]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId, 
r.c_cic_score ,
r.ScoreRange,
       ln_loan_level_user_type, flg_zero_non_granted_ever,
       flg_zero_granted_ever,
       Personal_Loans_granted_contracts_amt_24M,
       granted_contracts_cnt_6M, total_overdue_granted_contracts,
       has_ever_been_overdue, cnt_nongranted_contracts_3M,
       cnt_active_contracts, max_amt_granted_24M,
       tot_active_contracts_util, days_since_last_closed,
       vel_contract_nongranted_cnt_6on12,
       vel_contract_granted_amt_6on12,
       vel_contract_closed_amt_3on12,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType
from worktable_data_analysis.sil_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 3'
and r.c_cic_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-09-01' and '2025-01-31'
 >= '2024-09-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 923378fa-b060-4311-864e-dea5d11220b9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (11515, 20)


In [91]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'c_cic_score', 'ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12', 'osType'],
      dtype='object')

In [92]:
data.head()

,customer_id,digitalLoanAccountId,c_cic_score,ScoreRange,ln_loan_level_user_type,flg_zero_non_granted_ever,flg_zero_granted_ever,Personal_Loans_granted_contracts_amt_24M,granted_contracts_cnt_6M,total_overdue_granted_contracts,has_ever_been_overdue,cnt_nongranted_contracts_3M,cnt_active_contracts,max_amt_granted_24M,tot_active_contracts_util,days_since_last_closed,vel_contract_nongranted_cnt_6on12,vel_contract_granted_amt_6on12,vel_contract_closed_amt_3on12,osType
0,2274790,4d5d34e1-759e-4708-ba2c-5a9fbe277cee,0.153611,Ai,1_Repeat Applicant,0,0,NaN,NaN,0.0,0.0,NaN,NaN,8399.0,NaN,81.0,NaN,NaN,1.0,android
1,2833827,be54625c-f7a0-408e-8eca-69113d95ad74,0.145189,NH_Di,1_Repeat Applicant,1,0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,807.0,NaN,NaN,NaN,ios
2,3296391,b6fdd970-febd-407c-b894-57e02277e12f,0.101050,NH_Ii,1_Repeat Applicant,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,ios
3,3409304,6d2e297f-eed1-468d-b600-3c2122316aa3,0.270546,Ai,1_Repeat Applicant,0,0,23500.0,NaN,1.0,1.0,1.0,2.0,32942.0,0.929357,6.0,2.012048,NaN,1.0,android
4,2525100,a28d1147-4e88-4bdb-925b-bd9a2a8fd142,0.132305,Ai,1_Repeat Applicant,0,0,NaN,NaN,0.0,0.0,NaN,1.0,5999.0,0.371062,NaN,NaN,NaN,NaN,android


In [93]:
feature_column = ['ScoreRange',
       'ln_loan_level_user_type', 'flg_zero_non_granted_ever',
       'flg_zero_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'granted_contracts_cnt_6M', 'total_overdue_granted_contracts',
       'has_ever_been_overdue', 'cnt_nongranted_contracts_3M',
       'cnt_active_contracts', 'max_amt_granted_24M',
       'tot_active_contracts_util', 'days_since_last_closed',
       'vel_contract_nongranted_cnt_6on12', 'vel_contract_granted_amt_6on12',
       'vel_contract_closed_amt_3on12',]

dfd = transform_datav2(data, feature_column, a='c_cic_score', modelDisplayName='cic_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2274790,4d5d34e1-759e-4708-ba2c-5a9fbe277cee,9388855c-806c-4788-8160-77a0cab19f40,0.153611,2025-12-23T22:29:16.132759,2025-12-23T22:29:16.132759,cic_model_sil,v2,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Cash November 25 Models,6dfe86fa-9df6-403b-bc78-3705d6e8bcbb,2025-12-23T22:29:16.132759,{},Trench 3,android
1,2833827,be54625c-f7a0-408e-8eca-69113d95ad74,0a319884-2b1e-47c6-b194-5554d359de62,0.145189,2025-12-23T22:29:16.133754,2025-12-23T22:29:16.133754,cic_model_sil,v2,"{""ScoreRange"": ""NH_Di"", ""ln_loan_level_user_ty...",Cash November 25 Models,b254d390-15c5-41b8-af96-e154be9628ba,2025-12-23T22:29:16.133754,{},Trench 3,ios
2,3296391,b6fdd970-febd-407c-b894-57e02277e12f,8e61db62-ea02-4826-810b-0cf21b2559e5,0.101050,2025-12-23T22:29:16.133754,2025-12-23T22:29:16.133754,cic_model_sil,v2,"{""ScoreRange"": ""NH_Ii"", ""ln_loan_level_user_ty...",Cash November 25 Models,7dda5b19-740f-409a-bdd6-18f694b4fa96,2025-12-23T22:29:16.133754,{},Trench 3,ios
3,3409304,6d2e297f-eed1-468d-b600-3c2122316aa3,daf27073-f41f-45b9-8ac7-b78db3e3d3f2,0.270546,2025-12-23T22:29:16.133754,2025-12-23T22:29:16.133754,cic_model_sil,v2,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Cash November 25 Models,b9d48cf9-ade6-40c3-a9c3-e22d4ddb2c8b,2025-12-23T22:29:16.133754,{},Trench 3,android
4,2525100,a28d1147-4e88-4bdb-925b-bd9a2a8fd142,050079ef-14ef-45f0-ab54-05afd502a9a2,0.132305,2025-12-23T22:29:16.133754,2025-12-23T22:29:16.133754,cic_model_sil,v2,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Cash November 25 Models,4e6b3c2d-08b3-4f04-8ff0-659c1dc972fc,2025-12-23T22:29:16.133754,{},Trench 3,android


In [94]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0e0c8a28-61a5-40d2-8fc4-0b702e53c5c9>

##### Alpha Sil Stack Model 

##### Trench 1

In [95]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sa_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
r.sa_cic_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and sa_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-09-01' and '2025-02-28'
 >= '2024-09-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 47ba1766-e3c2-4bd2-b79b-a7a80451b1a4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (232107, 9)


In [96]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sa_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',
       'osType', 'ln_loan_type'],
      dtype='object')

In [97]:
data.head()

,customer_id,digitalLoanAccountId,sa_stack_score,sb_demo_score,apps_score,s_credo_score,sa_cic_score,osType,ln_loan_type
0,3724734,0e000dd0-5483-440f-a3a4-c780a8858fc5,0.286951,0.315114,NaN,0.092302,0.245948,ios,SIL Competitor
1,3728006,ae5706ba-9035-446c-b9ee-35198f9cdd21,0.166233,0.222729,0.399410,0.057848,0.227156,android,SIL Competitor
2,3663529,d51014b2-17e6-4bc4-b4b9-121b9de9af1f,0.190155,0.405539,0.358609,0.103233,0.125608,android,SIL Competitor
3,3624535,1e694dc1-044d-42e0-9c8a-5902042e0f18,0.301591,0.371884,0.385903,0.206535,0.209940,android,SIL Competitor
4,3527125,1f09abc4-b6e0-4973-ba19-621553888954,0.402495,0.301594,0.562777,0.109044,0.310822,android,SIL Competitor


In [98]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',]

dfd = transform_datav2(data, feature_column, a='sa_stack_score', modelDisplayName='alpha_stack_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3724734,0e000dd0-5483-440f-a3a4-c780a8858fc5,91943c03-2b89-42c9-807e-8579d0ad00e5,0.286951,2025-12-23T22:29:26.394789,2025-12-23T22:29:26.394789,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.31511421025365366, ""s_cred...",Cash November 25 Models,a5191257-37a7-494f-9b4d-4f872eb5c830,2025-12-23T22:29:26.394789,{},Trench 1,ios
1,3728006,ae5706ba-9035-446c-b9ee-35198f9cdd21,bbb4b9a7-a33c-4552-858c-fc3780dacb43,0.166233,2025-12-23T22:29:26.394789,2025-12-23T22:29:26.394789,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.2227289068986598, ""apps_sc...",Cash November 25 Models,992fe923-5ba1-4ed3-b333-52d4501b0baf,2025-12-23T22:29:26.394789,{},Trench 1,android
2,3663529,d51014b2-17e6-4bc4-b4b9-121b9de9af1f,2e52afe9-d149-4396-ac3e-1cd4171e065e,0.190155,2025-12-23T22:29:26.394789,2025-12-23T22:29:26.394789,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.4055388807530385, ""apps_sc...",Cash November 25 Models,0effc751-ab19-4900-be05-fb5aebc209dc,2025-12-23T22:29:26.394789,{},Trench 1,android
3,3624535,1e694dc1-044d-42e0-9c8a-5902042e0f18,498e928e-587f-45d4-a44f-6cbdbe151af4,0.301591,2025-12-23T22:29:26.394789,2025-12-23T22:29:26.394789,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.3718844634522161, ""apps_sc...",Cash November 25 Models,2f0fb1cf-28c7-48aa-a81e-cc7ffffebe5a,2025-12-23T22:29:26.394789,{},Trench 1,android
4,3527125,1f09abc4-b6e0-4973-ba19-621553888954,ca163268-3599-4490-8155-0b44d5b8a468,0.402495,2025-12-23T22:29:26.394789,2025-12-23T22:29:26.394789,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.30159383305909876, ""apps_s...",Cash November 25 Models,278df023-fc20-41b8-b2ce-179b18a3a782,2025-12-23T22:29:26.394789,{},Trench 1,android


In [99]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e7ba542f-88e1-44e8-96bc-c16b0feec477>

##### Trench 2

In [100]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sa_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
r.sa_cic_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and sa_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-09-01' and '2025-02-28'
 >= '2024-09-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 57498214-55cf-4fd8-a400-2a1c44de53eb successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (10592, 9)


In [101]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sa_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',
       'osType', 'ln_loan_type'],
      dtype='object')

In [102]:
data.head()

,customer_id,digitalLoanAccountId,sa_stack_score,sb_demo_score,apps_score,s_credo_score,sa_cic_score,osType,ln_loan_type
0,3293130,2cb4090d-5852-49c4-affa-53dfb7dfe288,0.648521,0.474293,0.670071,0.167973,0.259855,android,SIL Competitor
1,2392808,070eae26-e699-4ae8-80d5-c3e8214129d9,0.151161,0.308594,0.339616,0.056470,0.136852,android,SIL Competitor
2,3296339,2e765c88-e76f-4634-8c8f-e9f52ffb9128,0.632560,0.537717,0.444029,0.079980,0.469559,android,SIL Competitor
3,2875512,7ca5d2c9-71e8-4ab4-88fb-3dc14d96ef9f,0.431411,0.539373,0.412576,0.093245,0.258063,android,SIL Competitor
4,3317881,a7d9b9cd-b97c-4f54-abee-48695509918f,0.722003,0.636731,0.581282,0.097725,0.365066,android,SIL Competitor


In [103]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',]

dfd = transform_datav2(data, feature_column, a='sa_stack_score', modelDisplayName='alpha_stack_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3293130,2cb4090d-5852-49c4-affa-53dfb7dfe288,35d939ae-2626-4cea-a4fb-42fddeeedf2a,0.648521,2025-12-23T22:29:57.935959,2025-12-23T22:29:57.935959,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.4742933081455127, ""apps_sc...",Cash November 25 Models,4384256c-6e95-4543-b53b-ab7564e9f040,2025-12-23T22:29:57.935959,{},Trench 2,android
1,2392808,070eae26-e699-4ae8-80d5-c3e8214129d9,328700b9-8434-4c2f-a78a-2a16ae0af832,0.151161,2025-12-23T22:29:57.935959,2025-12-23T22:29:57.935959,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.308593926121455, ""apps_sco...",Cash November 25 Models,25b55ecc-e121-4ee9-a6f7-a30eedd7cc22,2025-12-23T22:29:57.935959,{},Trench 2,android
2,3296339,2e765c88-e76f-4634-8c8f-e9f52ffb9128,b38653d7-097d-4d01-b88c-2cf027942b33,0.632560,2025-12-23T22:29:57.935959,2025-12-23T22:29:57.935959,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.5377167750533154, ""apps_sc...",Cash November 25 Models,1c52343d-d575-4494-bccb-b13735ee07a9,2025-12-23T22:29:57.935959,{},Trench 2,android
3,2875512,7ca5d2c9-71e8-4ab4-88fb-3dc14d96ef9f,ad6efb05-5bfe-4dba-a0e8-affd5a9ed8a0,0.431411,2025-12-23T22:29:57.935959,2025-12-23T22:29:57.935959,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.539372592532463, ""apps_sco...",Cash November 25 Models,cb989ee4-f99d-407a-b1da-8f89ca84e14d,2025-12-23T22:29:57.935959,{},Trench 2,android
4,3317881,a7d9b9cd-b97c-4f54-abee-48695509918f,7a7ebda9-ee10-461a-a0b6-28aef23777d6,0.722003,2025-12-23T22:29:57.935959,2025-12-23T22:29:57.935959,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.636731153184002, ""apps_sco...",Cash November 25 Models,b6170b9c-0bd0-45a8-89bf-0015eafb71f0,2025-12-23T22:29:57.935959,{},Trench 2,android


In [104]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6ab04081-a94d-48cf-af2a-71f4fe6c605b>

##### Trench 3

In [105]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sa_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
r.sa_cic_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from worktable_data_analysis.sil_alpha_applied_loans_backscored_20240901_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 3'
and sa_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-09-01' and '2025-02-28'
 >= '2024-09-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID dd228431-6c4a-475d-bcf2-afc86a209327 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (11874, 9)


In [106]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sa_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',
       'osType', 'ln_loan_type'],
      dtype='object')

In [107]:
data.head()

,customer_id,digitalLoanAccountId,sa_stack_score,sb_demo_score,apps_score,s_credo_score,sa_cic_score,osType,ln_loan_type
0,2457664,3afbec05-c5f5-49a7-8d98-f4ae747c5d56,0.314217,0.316354,0.453302,0.063817,0.237723,android,SIL Competitor
1,2989813,ae562025-7ec3-46cb-9de6-733791a05199,0.455565,0.435061,0.535357,0.151845,0.161609,android,SIL Competitor
2,3111011,7aa952cb-b22a-4f6b-9bdd-1672d5638958,0.464130,0.386325,0.537683,0.088797,0.261037,android,SIL Competitor
3,3614141,4ca18655-1a6e-4752-aaf1-79e5ccdf1aec,0.214884,0.187046,0.441760,0.137558,0.146183,android,SIL Competitor
4,2536858,378664f3-ed97-489e-856d-470670920512,0.171608,0.194753,0.341388,0.139566,0.152214,android,SIL Competitor


In [108]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score', 'sa_cic_score',]

dfd = transform_datav2(data, feature_column, a='sa_stack_score', modelDisplayName='alpha_stack_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2457664,3afbec05-c5f5-49a7-8d98-f4ae747c5d56,55d2e780-9623-4034-9938-35d3140ffe5d,0.314217,2025-12-23T22:30:06.397284,2025-12-23T22:30:06.397284,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.316353802816822, ""apps_sco...",Cash November 25 Models,6975bb08-70bb-4fdd-990a-401098c4f374,2025-12-23T22:30:06.397284,{},Trench 3,android
1,2989813,ae562025-7ec3-46cb-9de6-733791a05199,8c18e183-0f0b-4e6f-8079-ae5ae4310c80,0.455565,2025-12-23T22:30:06.397284,2025-12-23T22:30:06.397284,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.43506135183303596, ""apps_s...",Cash November 25 Models,1158d2ee-6fe4-46e6-a8b3-41830c227bff,2025-12-23T22:30:06.397284,{},Trench 3,android
2,3111011,7aa952cb-b22a-4f6b-9bdd-1672d5638958,8fc5555c-5c59-421d-a52b-b6c58f06ad3c,0.464130,2025-12-23T22:30:06.398283,2025-12-23T22:30:06.398283,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.3863248978061694, ""apps_sc...",Cash November 25 Models,ee3e16ad-9034-4f35-8947-5f0bd401513e,2025-12-23T22:30:06.398283,{},Trench 3,android
3,3614141,4ca18655-1a6e-4752-aaf1-79e5ccdf1aec,de2ccf0f-bda6-42a5-8dbe-ef59a6801bf4,0.214884,2025-12-23T22:30:06.398283,2025-12-23T22:30:06.398283,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.18704637976505253, ""apps_s...",Cash November 25 Models,ec9b33a3-204f-4820-925b-1865c21de3ba,2025-12-23T22:30:06.398283,{},Trench 3,android
4,2536858,378664f3-ed97-489e-856d-470670920512,a4bb8d2e-5a45-4d68-b9bf-bfd78e56ba12,0.171608,2025-12-23T22:30:06.398283,2025-12-23T22:30:06.398283,alpha_stack_model_sil,v2,"{""sb_demo_score"": 0.19475266563777466, ""apps_s...",Cash November 25 Models,2a5b38c1-02db-4450-bdd7-948132618b12,2025-12-23T22:30:06.398283,{},Trench 3,android


In [109]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0b0899dd-9a99-4845-9831-e4835b7b4dd2>

##### Beta SIL STACK Score Model

##### Trench 1

In [110]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sb_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from prj-prod-dataplatform.worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1'
and sb_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-08-01' and '2025-02-28'
 >= '2024-08-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 95fe1dfa-ee86-4b0a-a5c1-5aa03470ce7f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (343968, 8)


In [111]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sb_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'osType',
       'ln_loan_type'],
      dtype='object')

In [112]:
data.head()

,customer_id,digitalLoanAccountId,sb_stack_score,sb_demo_score,apps_score,s_credo_score,osType,ln_loan_type
0,3706433,093b3b88-b80f-452f-b827-fe2579a27234,0.317903,0.347730,0.353075,0.190357,android,SIL Competitor
1,3718797,0b62d2dd-f047-4436-a248-3ab8eb894712,0.348431,0.410788,0.447922,0.072309,android,SIL Competitor
2,3337602,0378069b-985a-474d-9c76-199085d980c5,0.193233,0.332930,0.296256,0.071188,android,SIL Competitor
3,3640791,27356bab-540f-46eb-b31b-98ad4d06da64,0.142487,0.280734,0.256432,0.056010,android,SIL Competitor
4,3310515,28348a8a-88dd-4b73-bd2b-f3bb556e2047,0.426251,0.410245,0.533476,0.078765,android,SIL Competitor


In [113]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score']

dfd = transform_datav2(data, feature_column, a='sb_stack_score', modelDisplayName='beta_stack_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3706433,093b3b88-b80f-452f-b827-fe2579a27234,6ddee1ab-f6b6-4c2d-a345-acdd9a4aad5a,0.317903,2025-12-23T22:30:17.285041,2025-12-23T22:30:17.285041,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.3477298227561125, ""apps_sc...",Cash November 25 Models,7dfd39b2-1c21-43a7-8614-bbccde75db62,2025-12-23T22:30:17.285041,{},Trench 1,android
1,3718797,0b62d2dd-f047-4436-a248-3ab8eb894712,7a693b14-3472-44e8-af46-5fc108b0c676,0.348431,2025-12-23T22:30:17.285600,2025-12-23T22:30:17.285600,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.4107883971102936, ""apps_sc...",Cash November 25 Models,d610417d-98d8-40b0-952f-cbdcc827eb53,2025-12-23T22:30:17.285600,{},Trench 1,android
2,3337602,0378069b-985a-474d-9c76-199085d980c5,c98dd0df-422a-412d-b3e0-3b6340480587,0.193233,2025-12-23T22:30:17.285600,2025-12-23T22:30:17.285600,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.3329297011213244, ""apps_sc...",Cash November 25 Models,15e5e741-351e-43a8-a151-57b3500ba0f5,2025-12-23T22:30:17.285600,{},Trench 1,android
3,3640791,27356bab-540f-46eb-b31b-98ad4d06da64,1956089f-166f-42d2-9bc8-134882889a06,0.142487,2025-12-23T22:30:17.285600,2025-12-23T22:30:17.285600,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.2807343719378874, ""apps_sc...",Cash November 25 Models,52ad8f1f-e402-4f50-8cdc-8c8e81ddd2b6,2025-12-23T22:30:17.285600,{},Trench 1,android
4,3310515,28348a8a-88dd-4b73-bd2b-f3bb556e2047,dfa5e1a9-4601-4d6b-8661-1745b94c0608,0.426251,2025-12-23T22:30:17.285600,2025-12-23T22:30:17.285600,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.4102450106935408, ""apps_sc...",Cash November 25 Models,dff58924-678e-4f06-9065-5d152dceb110,2025-12-23T22:30:17.285600,{},Trench 1,android


In [114]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=97c7e48f-663f-4c04-9edc-0a372a64b746>

##### Trench 2

In [115]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sb_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from prj-prod-dataplatform.worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2'
and sb_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-08-01' and '2025-02-28'
 >= '2024-08-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 1c44a765-f4ae-4179-b383-662270b63691 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13099, 8)


In [116]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sb_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'osType',
       'ln_loan_type'],
      dtype='object')

In [117]:
data.head()

,customer_id,digitalLoanAccountId,sb_stack_score,sb_demo_score,apps_score,s_credo_score,osType,ln_loan_type
0,2806685,1e3d935f-5a64-4b13-b152-7fffa06f69ae,0.707874,0.711040,0.459994,0.211854,android,SIL Competitor
1,2851742,6a9c389c-c8b4-4e29-b7aa-f719d0cd4ff6,0.796401,0.626172,0.616544,0.266893,android,SIL Competitor
2,2440120,5ac9287b-56eb-43e5-aa0a-29ea25640762,0.184507,0.340662,0.297374,0.043929,android,SIL Competitor
3,2718257,2236cd4e-1233-4370-8a24-2f63a3c1fa04,0.567638,0.369119,0.619353,0.189963,android,SIL Competitor
4,3225097,c64f2027-fbac-4b74-b76f-261a28e5b59a,0.252371,0.223552,NaN,0.081028,ios,SIL Competitor


In [118]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score']

dfd = transform_datav2(data, feature_column, a='sb_stack_score', modelDisplayName='beta_stack_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2806685,1e3d935f-5a64-4b13-b152-7fffa06f69ae,3c2f16ab-aaed-45e5-85f0-7de09a29e85e,0.707874,2025-12-23T22:30:58.747790,2025-12-23T22:30:58.747790,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.7110400254007136, ""apps_sc...",Cash November 25 Models,88bdb845-7afa-49f6-b1bb-4e247189f1fb,2025-12-23T22:30:58.747790,{},Trench 2,android
1,2851742,6a9c389c-c8b4-4e29-b7aa-f719d0cd4ff6,22373847-c25c-41de-86c8-a4b20fd21d45,0.796401,2025-12-23T22:30:58.747790,2025-12-23T22:30:58.747790,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.6261720970302769, ""apps_sc...",Cash November 25 Models,419bc742-6529-47fa-9b92-4e51de30c1c9,2025-12-23T22:30:58.747790,{},Trench 2,android
2,2440120,5ac9287b-56eb-43e5-aa0a-29ea25640762,ea544696-134a-4b85-b19c-8752ea6aec77,0.184507,2025-12-23T22:30:58.747790,2025-12-23T22:30:58.747790,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.3406622909960318, ""apps_sc...",Cash November 25 Models,65ee167d-8630-4edf-8b6c-f70f597cd276,2025-12-23T22:30:58.747790,{},Trench 2,android
3,2718257,2236cd4e-1233-4370-8a24-2f63a3c1fa04,a2a850ee-0c62-488e-9ce1-67e7cf148613,0.567638,2025-12-23T22:30:58.747790,2025-12-23T22:30:58.747790,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.3691194725557818, ""apps_sc...",Cash November 25 Models,90f53f2b-5514-422d-b55b-fb63f943686a,2025-12-23T22:30:58.747790,{},Trench 2,android
4,3225097,c64f2027-fbac-4b74-b76f-261a28e5b59a,6187cb41-9aca-43ba-b662-70458ee8f46d,0.252371,2025-12-23T22:30:58.747790,2025-12-23T22:30:58.747790,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.22355239842161287, ""s_cred...",Cash November 25 Models,17782881-3339-4aae-b4ea-2d906a57a5d8,2025-12-23T22:30:58.747790,{},Trench 2,ios


In [119]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c165ad06-8e3f-4395-9213-2f69e94f0697>

##### Trench 3

In [120]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,  
r.sb_stack_score ,
r.sb_demo_score,
r.apps_score,
r.s_credo_score,
case 
  when lower(r.ln_os_type) like '%andro%' then 'android'
  when lower(r.ln_os_type) like '%os%' then 'ios'
  else 'ios' end  osType,
ln_loan_type
from prj-prod-dataplatform.worktable_data_analysis.sil_beta_applied_loans_backscored_20240801_20251013_option3 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 3'
and sb_stack_score is not null
and IF(lmt.new_loan_type = 'Flex-up', lmt.startApplyDateTime, lmt.termsAndConditionsSubmitDateTime) 
 -- between '2024-08-01' and '2025-02-28'
 >= '2024-08-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 5bf4d9f1-0c9c-4dd0-9768-f4b3d8353a34 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13802, 8)


In [121]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'sb_stack_score',
       'sb_demo_score', 'apps_score', 's_credo_score', 'osType',
       'ln_loan_type'],
      dtype='object')

In [122]:
data.head()

,customer_id,digitalLoanAccountId,sb_stack_score,sb_demo_score,apps_score,s_credo_score,osType,ln_loan_type
0,2787972,1de3a0cb-8da3-4516-839b-5e3f0e94bfe9,0.201704,0.184555,0.428684,0.083273,android,SIL Competitor
1,2819148,36849de5-183a-4d79-bab8-ec1ee5bfe8e1,0.382731,0.209561,0.630260,0.109346,android,SIL Competitor
2,2976230,5f1d56f8-cf9b-40c1-b004-b91aa9bb5a66,0.223757,0.283909,0.358850,0.100727,android,SIL Competitor
3,3353627,75b58340-9fda-4137-bf91-2593daed1172,0.079870,0.201554,0.155217,0.043275,android,SIL Competitor
4,2626027,31506333-f284-4d8a-adf4-6c4a7bf1333d,0.115679,0.190968,0.226612,0.097335,android,SIL Competitor


In [123]:
feature_column = ['sb_demo_score', 'apps_score', 's_credo_score']

dfd = transform_datav2(data, feature_column, a='sb_stack_score', modelDisplayName='beta_stack_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2787972,1de3a0cb-8da3-4516-839b-5e3f0e94bfe9,02aab962-503c-464c-ba55-ba2658ddfcba,0.201704,2025-12-23T22:31:05.532437,2025-12-23T22:31:05.532437,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.18455473473617623, ""apps_s...",Cash November 25 Models,aa318a00-7ba4-4c4c-9da8-f8d4a6360e5c,2025-12-23T22:31:05.532437,{},Trench 3,android
1,2819148,36849de5-183a-4d79-bab8-ec1ee5bfe8e1,34de6e2f-84fc-44e7-a9e5-7575ce3e9986,0.382731,2025-12-23T22:31:05.532979,2025-12-23T22:31:05.532979,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.20956102253551195, ""apps_s...",Cash November 25 Models,cb609eef-25a5-49f1-8bae-556e01d6fda9,2025-12-23T22:31:05.532979,{},Trench 3,android
2,2976230,5f1d56f8-cf9b-40c1-b004-b91aa9bb5a66,5c26d9c6-09c9-4715-b91e-486057ee52d2,0.223757,2025-12-23T22:31:05.532979,2025-12-23T22:31:05.532979,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.28390921046765677, ""apps_s...",Cash November 25 Models,6e5d3d85-ffb7-4cb1-97e0-6b1881f6bf5c,2025-12-23T22:31:05.532979,{},Trench 3,android
3,3353627,75b58340-9fda-4137-bf91-2593daed1172,3c8f95df-5867-46bf-a29e-4ecc39cc7c18,0.079870,2025-12-23T22:31:05.532979,2025-12-23T22:31:05.532979,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.20155387836445485, ""apps_s...",Cash November 25 Models,8afc5484-c2c4-4666-a2be-898ab70f1938,2025-12-23T22:31:05.532979,{},Trench 3,android
4,2626027,31506333-f284-4d8a-adf4-6c4a7bf1333d,f4e56361-55bf-461c-a9c2-1faf94032028,0.115679,2025-12-23T22:31:05.532979,2025-12-23T22:31:05.532979,beta_stack_model_sil,v2,"{""sb_demo_score"": 0.1909678221022105, ""apps_sc...",Cash November 25 Models,ce920c44-9b02-4606-9183-1df9beefcbe8,2025-12-23T22:31:05.532979,{},Trench 3,android


In [124]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=549ac876-9991-4778-ab27-82a5e983ef36>

##### Beta Sil App Score

##### Trench 1

In [125]:
sq = """ 
select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.apps_score,
app_cnt_payday_ever,
app_cnt_finance_ever,
app_cnt_competitors_sil_ever,
app_cnt_competitors_ever,
app_cnt_finance_365d,
app_cnt_absence_tag_365d,
app_cnt_competitors_sil_365d,
app_cnt_finance_7d,
app_cnt_rated_for_3plus_ever,
app_cnt_payday_7d,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from  risk_mart.applied_sil_new_applicants_loan_20241001_20251124_app_scored r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and trench_category = 'Trench 1'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
 -- between '2024-10-01' and '2025-11-24'
 >= '2024-10-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 4ffac054-5a84-4c4f-b4ac-509ca7e26624 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (296743, 14)


In [126]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'apps_score',
       'app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d', 'osType'],
      dtype='object')

In [127]:
data.head()

,customer_id,digitalLoanAccountId,apps_score,app_cnt_payday_ever,app_cnt_finance_ever,app_cnt_competitors_sil_ever,app_cnt_competitors_ever,app_cnt_finance_365d,app_cnt_absence_tag_365d,app_cnt_competitors_sil_365d,app_cnt_finance_7d,app_cnt_rated_for_3plus_ever,app_cnt_payday_7d,osType
0,3374641,2eb5851c-9f78-466b-84fd-80796280168c,0.539085,4,13.0,2,7,13.0,14.0,2.0,1.0,37.0,0.0,android
1,3797449,9ab3c26d-bd51-4aed-9d43-8b135430794b,0.351878,7,14.0,3,2,9.0,13.0,2.0,0.0,64.0,0.0,android
2,3795450,ba7bc09c-6c6c-4284-a3b7-544e495aab59,0.575629,9,17.0,1,6,6.0,15.0,1.0,0.0,53.0,0.0,android
3,3694589,5b5b6f4d-4cd7-41a9-8d0e-a7a98dad9b72,0.308280,3,17.0,3,8,11.0,7.0,2.0,2.0,44.0,2.0,android
4,3572219,6d92a886-17ea-45b7-a214-5888f50b8415,0.328260,6,13.0,3,5,9.0,12.0,2.0,1.0,52.0,1.0,android


In [128]:
feature_column = ['app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d',]

dfd = transform_datav2(data, feature_column, a='apps_score', modelDisplayName='apps_score_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3374641,2eb5851c-9f78-466b-84fd-80796280168c,c35afb45-5a16-4e07-922c-7a0d3fecad2f,0.539085,2025-12-23T22:31:17.018252,2025-12-23T22:31:17.018252,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 4, ""app_cnt_finance_ev...",Cash November 25 Models,ba66a22b-5b66-41ed-bc5a-b983d6e8199e,2025-12-23T22:31:17.018252,{},Trench 1,android
1,3797449,9ab3c26d-bd51-4aed-9d43-8b135430794b,70f7e73e-67d3-4993-8cfd-1cacf5e72cc1,0.351878,2025-12-23T22:31:17.019254,2025-12-23T22:31:17.019254,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 7, ""app_cnt_finance_ev...",Cash November 25 Models,0bc4e393-1680-4741-9140-5042087a3055,2025-12-23T22:31:17.019254,{},Trench 1,android
2,3795450,ba7bc09c-6c6c-4284-a3b7-544e495aab59,1a25f6d5-f1a7-47b6-a1fe-9d3ac60c216a,0.575629,2025-12-23T22:31:17.019254,2025-12-23T22:31:17.019254,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 9, ""app_cnt_finance_ev...",Cash November 25 Models,99d01e8c-7a81-4930-a17c-12ce127c9292,2025-12-23T22:31:17.019254,{},Trench 1,android
3,3694589,5b5b6f4d-4cd7-41a9-8d0e-a7a98dad9b72,5a2270f5-e021-48eb-8cad-4dd418feeeea,0.308280,2025-12-23T22:31:17.019254,2025-12-23T22:31:17.019254,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 3, ""app_cnt_finance_ev...",Cash November 25 Models,0404087d-ad8e-4dc7-84e3-c787002bb8cb,2025-12-23T22:31:17.019254,{},Trench 1,android
4,3572219,6d92a886-17ea-45b7-a214-5888f50b8415,af53c72a-e9d9-4cb3-8328-1c23a8cba0b1,0.328260,2025-12-23T22:31:17.019254,2025-12-23T22:31:17.019254,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 6, ""app_cnt_finance_ev...",Cash November 25 Models,23883bfc-e91f-4fa0-a3f1-7bf4a19489b1,2025-12-23T22:31:17.019254,{},Trench 1,android


In [129]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=18d282ed-e281-4808-8b6b-64a76f5b7a57>

##### Trench 2

In [130]:
sq = """ 
select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.apps_score,
app_cnt_payday_ever,
app_cnt_finance_ever,
app_cnt_competitors_sil_ever,
app_cnt_competitors_ever,
app_cnt_finance_365d,
app_cnt_absence_tag_365d,
app_cnt_competitors_sil_365d,
app_cnt_finance_7d,
app_cnt_rated_for_3plus_ever,
app_cnt_payday_7d,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from  risk_mart.applied_sil_new_applicants_loan_20241001_20251124_app_scored r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and trench_category = 'Trench 2'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
 -- between '2024-10-01' and '2025-11-24'
 >= '2024-10-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 3aee6d21-aafa-44e2-8576-81db7b7859f6 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (12240, 14)


In [131]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'apps_score',
       'app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d', 'osType'],
      dtype='object')

In [132]:
data.head()

,customer_id,digitalLoanAccountId,apps_score,app_cnt_payday_ever,app_cnt_finance_ever,app_cnt_competitors_sil_ever,app_cnt_competitors_ever,app_cnt_finance_365d,app_cnt_absence_tag_365d,app_cnt_competitors_sil_365d,app_cnt_finance_7d,app_cnt_rated_for_3plus_ever,app_cnt_payday_7d,osType
0,1048822,38ce1a66-7d2b-44d3-923c-8d3fd9ed9287,0.202982,6,29.0,3,13,29.0,20.0,3.0,0.0,93.0,0.0,android
1,3006119,3aba6ad7-0b98-45ad-b88b-5ada4a6b8af2,0.585864,15,18.0,2,6,18.0,30.0,2.0,0.0,45.0,0.0,android
2,2968404,348b4f8e-c8f4-450a-8aa8-e7e665ac8412,0.464053,6,24.0,1,9,13.0,12.0,1.0,0.0,72.0,0.0,android
3,2316774,a1c9608e-b2ba-4462-ac1e-e2d0d60155c9,0.517704,3,22.0,3,10,22.0,37.0,3.0,0.0,61.0,0.0,android
4,1277623,4d3c9a62-880e-49d2-b320-73d5eaf3bf50,0.278553,9,21.0,3,11,17.0,8.0,2.0,0.0,71.0,0.0,android


In [133]:
feature_column = ['app_cnt_payday_ever', 'app_cnt_finance_ever',
       'app_cnt_competitors_sil_ever', 'app_cnt_competitors_ever',
       'app_cnt_finance_365d', 'app_cnt_absence_tag_365d',
       'app_cnt_competitors_sil_365d', 'app_cnt_finance_7d',
       'app_cnt_rated_for_3plus_ever', 'app_cnt_payday_7d',]

dfd = transform_datav2(data, feature_column, a='apps_score', modelDisplayName='apps_score_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1048822,38ce1a66-7d2b-44d3-923c-8d3fd9ed9287,64134bd0-a8c2-4235-8aef-aad0336017a5,0.202982,2025-12-23T22:32:14.195063,2025-12-23T22:32:14.195063,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 6, ""app_cnt_finance_ev...",Cash November 25 Models,7b0e848f-4221-4ea8-b10c-d9c3d697e8a1,2025-12-23T22:32:14.195063,{},Trench 2,android
1,3006119,3aba6ad7-0b98-45ad-b88b-5ada4a6b8af2,cdad377d-b116-45f1-b78f-b87c7f812801,0.585864,2025-12-23T22:32:14.195063,2025-12-23T22:32:14.195063,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 15, ""app_cnt_finance_e...",Cash November 25 Models,1e6d9bf3-3572-46af-b23f-ea90524b7f5d,2025-12-23T22:32:14.195063,{},Trench 2,android
2,2968404,348b4f8e-c8f4-450a-8aa8-e7e665ac8412,3bc31d2e-0811-42f8-a432-a2c032f60b6f,0.464053,2025-12-23T22:32:14.195063,2025-12-23T22:32:14.195063,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 6, ""app_cnt_finance_ev...",Cash November 25 Models,ef9f9500-a23c-45e6-8961-cfccfdf359c5,2025-12-23T22:32:14.195063,{},Trench 2,android
3,2316774,a1c9608e-b2ba-4462-ac1e-e2d0d60155c9,4d75c3a9-08a9-4924-984b-2b57886c60ee,0.517704,2025-12-23T22:32:14.196062,2025-12-23T22:32:14.196062,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 3, ""app_cnt_finance_ev...",Cash November 25 Models,0dfc58d2-6b9c-419d-9092-17be95fcc3ba,2025-12-23T22:32:14.196062,{},Trench 2,android
4,1277623,4d3c9a62-880e-49d2-b320-73d5eaf3bf50,c618b140-9de0-49ab-ab95-610e45d697c9,0.278553,2025-12-23T22:32:14.196062,2025-12-23T22:32:14.196062,apps_score_model_sil,v2,"{""app_cnt_payday_ever"": 9, ""app_cnt_finance_ev...",Cash November 25 Models,a3fc5f3f-b38c-4a41-ac99-8b906279025f,2025-12-23T22:32:14.196062,{},Trench 2,android


In [134]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0b05b901-1199-41ac-9250-61c44f178193>

##### Trench 3

In [135]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.apps_score,
app_cnt_productivity_ever,
app_cnt_rated_for_3plus_ever, 
app_cnt_books_and_reference_ever,
app_cnt_tools_ever, 
app_median_time_bw_installed_mins_3d,
app_median_time_bw_installed_mins_30d,
app_cnt_communication_ever, 
app_cnt_finance_90d,
app_cnt_absence_tag_180d, 
app_cnt_shopping_ever,
app_cnt_social_ever, 
app_cnt_driver_90d, 
app_cnt_payday_365d,
app_cnt_driver_365d, 
app_cnt_music_and_audio_ever,
app_cnt_finance_180d, 
app_cnt_art_and_design_ever,
app_cnt_gaming_90d, 
app_avg_time_bw_installed_mins_30d,
app_cnt_education_ever,
case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from risk_mart.applied_sil_repeat_applicants_loan_20241001_20251124_app_scored r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
 -- between '2024-10-01' and '2025-11-24'
 >= '2024-10-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 7031d687-74e8-4fd4-b680-a54389d30154 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (20246, 24)


In [136]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'apps_score',
       'app_cnt_productivity_ever', 'app_cnt_rated_for_3plus_ever',
       'app_cnt_books_and_reference_ever', 'app_cnt_tools_ever',
       'app_median_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_30d', 'app_cnt_communication_ever',
       'app_cnt_finance_90d', 'app_cnt_absence_tag_180d',
       'app_cnt_shopping_ever', 'app_cnt_social_ever', 'app_cnt_driver_90d',
       'app_cnt_payday_365d', 'app_cnt_driver_365d',
       'app_cnt_music_and_audio_ever', 'app_cnt_finance_180d',
       'app_cnt_art_and_design_ever', 'app_cnt_gaming_90d',
       'app_avg_time_bw_installed_mins_30d', 'app_cnt_education_ever',
       'osType'],
      dtype='object')

In [137]:
feature_column = ['app_cnt_productivity_ever', 'app_cnt_rated_for_3plus_ever',
       'app_cnt_books_and_reference_ever', 'app_cnt_tools_ever',
       'app_median_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_30d', 'app_cnt_communication_ever',
       'app_cnt_finance_90d', 'app_cnt_absence_tag_180d',
       'app_cnt_shopping_ever', 'app_cnt_social_ever', 'app_cnt_driver_90d',
       'app_cnt_payday_365d', 'app_cnt_driver_365d',
       'app_cnt_music_and_audio_ever', 'app_cnt_finance_180d',
       'app_cnt_art_and_design_ever', 'app_cnt_gaming_90d',
       'app_avg_time_bw_installed_mins_30d', 'app_cnt_education_ever',]

dfd = transform_datav2(data, feature_column, a='apps_score', modelDisplayName='apps_score_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2765697,7aa0f9e1-9a2a-480d-a9eb-4e1383bb3c11,10e034db-415d-4fcb-84ce-c259360a6764,0.298492,2025-12-23T22:32:22.260540,2025-12-23T22:32:22.260540,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 2.0, ""app_cnt_ra...",Cash November 25 Models,40be739e-0dd4-423d-8c8c-c9e97c4ebfdb,2025-12-23T22:32:22.260540,{},Trench 3,android
1,3428332,a7b10ee7-2900-44ed-b484-6b58b5433691,8d5cd332-440f-4657-b251-6c3a4f82f646,0.626622,2025-12-23T22:32:22.260540,2025-12-23T22:32:22.260540,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 0.0, ""app_cnt_ra...",Cash November 25 Models,6efb64e3-5fa3-495e-8c87-ab79df01f5a6,2025-12-23T22:32:22.260540,{},Trench 3,android
2,2517253,d0375de0-ba05-429d-93a3-b23761834d4f,8d2b75cd-5bc9-455d-a841-ea8f3bb56f79,0.384529,2025-12-23T22:32:22.260540,2025-12-23T22:32:22.260540,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 3.0, ""app_cnt_ra...",Cash November 25 Models,df7b9f8f-709b-4260-977e-dcf025c8cd49,2025-12-23T22:32:22.260540,{},Trench 3,android
3,2208512,f9ad0525-f7ed-4872-9dc8-f6f45b138a02,65534304-475a-42b7-97af-28bafda58ca6,0.225759,2025-12-23T22:32:22.261538,2025-12-23T22:32:22.261538,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 14.0, ""app_cnt_r...",Cash November 25 Models,d0f88034-8c71-4c75-bb0f-dea691923d69,2025-12-23T22:32:22.261538,{},Trench 3,android
4,2676174,ae912cea-0885-4efc-bc07-7ff0ca4a0456,5b51b4ea-a830-4482-884f-96fb3097c168,0.452701,2025-12-23T22:32:22.261538,2025-12-23T22:32:22.261538,apps_score_model_sil,v2,"{""app_cnt_productivity_ever"": 5.0, ""app_cnt_ra...",Cash November 25 Models,80f9190c-2b07-46b1-bb9d-9fc8a0269285,2025-12-23T22:32:22.261538,{},Trench 3,android


In [138]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [139]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20246 entries, 0 to 20245
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            20246 non-null  int64  
 1   digitalLoanAccountId  20246 non-null  object 
 2   crifApplicationId     20246 non-null  object 
 3   prediction            20246 non-null  float64
 4   start_time            20246 non-null  object 
 5   end_time              20246 non-null  object 
 6   modelDisplayName      20246 non-null  object 
 7   modelVersionId        20246 non-null  object 
 8   calcFeature           20246 non-null  object 
 9   subscription_name     20246 non-null  object 
 10  message_id            20246 non-null  object 
 11  publish_time          20246 non-null  object 
 12  attributes            20246 non-null  object 
 13  trenchCategory        20246 non-null  object 
 14  deviceOs              20246 non-null  object 
dtypes: float64(1), int6

In [140]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f593a270-ca99-46f2-8385-b8bc12d6dc6f>

##### Beta SIL Demo Score

##### Trench 1

In [141]:
sq = """select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.s_demo_score,
ln_vas_opted_flag,
ln_doc_type_rolled,
ln_industry_new_cat_bin,
ln_marital_status,
ln_age,
ln_education_level,
ln_cnt_dependents,
ln_ref2_type,
ln_loan_level_user_type,
ln_ref1_type,
ln_name_email_match_score,
ln_telconame,
ln_city_cat,
ln_brand_bin,
ln_apply_Is_Weekend,
r.ln_os_type osType
from worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where s_demo_score is not null
and trench_category = 'Trench 1'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
 -- between '2024-08-01' and '2025-01-31'
 >= '2024-08-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID c915ff9d-58b6-4937-925c-7e36f75acd95 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (345905, 19)


In [142]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 's_demo_score',
       'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType'],
      dtype='object')

In [143]:
feature_column = [ 'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType']

dfd = transform_datav2(data, feature_column, a='s_demo_score', modelDisplayName='beta_demo_model_sil', tc='Trench 1', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3611632,f47eb312-c566-4af4-b9eb-1b30b985a04f,91297721-03f8-4b7a-985f-f5f303ad2f7c,0.580767,2025-12-23T22:32:38.847465,2025-12-23T22:32:38.847465,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,e76b0eea-614f-4fd8-a19d-1adfd39f5c1e,2025-12-23T22:32:38.847465,{},Trench 1,Android
1,3089141,3c622928-6535-4fd3-b595-6cfe30c27023,88c9c6f8-1062-435a-b03a-c7151cd10f56,0.352142,2025-12-23T22:32:38.848465,2025-12-23T22:32:38.848465,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,b58a793b-1ba5-42aa-9fc4-a02dfa247f4d,2025-12-23T22:32:38.848465,{},Trench 1,Android
2,3225107,ad1c148b-1e45-498b-8715-5fb1f7d7d4aa,0c73768b-c6b5-4e22-9acc-9a4b63787d7b,0.446370,2025-12-23T22:32:38.848465,2025-12-23T22:32:38.848465,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,cf36d3fb-4a48-4335-baa3-fa056699d8e1,2025-12-23T22:32:38.848465,{},Trench 1,Android
3,3330039,acaf286d-f224-4bda-95f1-d30f1dfb6c3d,f3fe940e-3a60-4a87-b580-1a5e567e4d02,0.306876,2025-12-23T22:32:38.848465,2025-12-23T22:32:38.848465,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,da6164ab-7e05-4e48-8dc6-9ed6967b8dc5,2025-12-23T22:32:38.848465,{},Trench 1,Android
4,3429499,5725d1fa-23f3-4c08-966f-c51bd603a3a9,384bcd2b-09e6-41ca-bda9-784456b97b97,0.266357,2025-12-23T22:32:38.848465,2025-12-23T22:32:38.848465,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,4892883e-3042-4967-bcdf-be876cfe6d6d,2025-12-23T22:32:38.848465,{},Trench 1,Android


In [144]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [145]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345905 entries, 0 to 345904
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            345905 non-null  int64  
 1   digitalLoanAccountId  345905 non-null  object 
 2   crifApplicationId     345905 non-null  object 
 3   prediction            345905 non-null  float64
 4   start_time            345905 non-null  object 
 5   end_time              345905 non-null  object 
 6   modelDisplayName      345905 non-null  object 
 7   modelVersionId        345905 non-null  object 
 8   calcFeature           345905 non-null  object 
 9   subscription_name     345905 non-null  object 
 10  message_id            345905 non-null  object 
 11  publish_time          345905 non-null  object 
 12  attributes            345905 non-null  object 
 13  trenchCategory        345905 non-null  object 
 14  deviceOs              345905 non-null  object 
dtype

In [146]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=61fa0fc5-7327-4c33-a6f3-c555952cc312>

##### Trench 2

In [147]:
sq = """select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.s_demo_score,
ln_vas_opted_flag,
ln_doc_type_rolled,
ln_industry_new_cat_bin,
ln_marital_status,
ln_age,
ln_education_level,
ln_cnt_dependents,
ln_ref2_type,
ln_loan_level_user_type,
ln_ref1_type,
ln_name_email_match_score,
ln_telconame,
ln_city_cat,
ln_brand_bin,
ln_apply_Is_Weekend,
r.ln_os_type osType
from worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where s_demo_score is not null
and trench_category = 'Trench 2'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
 -- between '2024-08-01' and '2025-01-31'
 >= '2024-08-01'
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID a315bd2b-b5e7-4f56-95d8-8d56ba1490dc successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13192, 19)


In [148]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 's_demo_score',
       'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType'],
      dtype='object')

In [149]:
feature_column = [ 'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType']

dfd = transform_datav2(data, feature_column, a='s_demo_score', modelDisplayName='beta_demo_model_sil', tc='Trench 2', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2995155,ae062146-6702-46d1-a694-49ebba775caf,826dc4e0-1a33-448c-9aac-101cd5ef0bf3,0.269529,2025-12-23T22:33:58.937999,2025-12-23T22:33:58.937999,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,901f9d37-3e38-490e-b431-3404b1cd6bf2,2025-12-23T22:33:58.937999,{},Trench 2,Android
1,2275327,0eba084a-e278-4005-a9f5-5f4954d38e55,ffc35aac-b2f2-4a7e-bb58-910a535bcf1e,0.328075,2025-12-23T22:33:58.937999,2025-12-23T22:33:58.937999,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,f88fcac1-6eed-4f84-9791-e03369aeb1ca,2025-12-23T22:33:58.937999,{},Trench 2,Android
2,2937583,fa403542-38c6-433b-91f2-3bcb3fa3b1ef,8461f90d-fbd1-431b-b53c-401c881b386c,0.424187,2025-12-23T22:33:58.937999,2025-12-23T22:33:58.937999,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,eaa5a729-b413-4dbc-975e-b90a67a5e4b3,2025-12-23T22:33:58.937999,{},Trench 2,iOS
3,3314619,905dc14c-d185-4c6c-b9b4-4cea9147d7c5,c991bf21-5ee1-4f05-9b40-8c342a55fa0f,0.652294,2025-12-23T22:33:58.939000,2025-12-23T22:33:58.939000,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,e1632f1e-722a-442d-a072-0c094b14cad8,2025-12-23T22:33:58.939000,{},Trench 2,Android
4,2325245,beafcb7a-0029-44a5-a6d3-fd2728fe9aaa,edc40baa-ae8f-4a6d-8560-0441541c0766,0.371950,2025-12-23T22:33:58.939000,2025-12-23T22:33:58.939000,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",Cash November 25 Models,a0178af2-7783-437e-b9a3-e77694df7f2a,2025-12-23T22:33:58.939000,{},Trench 2,Android


In [150]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [151]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13192 entries, 0 to 13191
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            13192 non-null  int64  
 1   digitalLoanAccountId  13192 non-null  object 
 2   crifApplicationId     13192 non-null  object 
 3   prediction            13192 non-null  float64
 4   start_time            13192 non-null  object 
 5   end_time              13192 non-null  object 
 6   modelDisplayName      13192 non-null  object 
 7   modelVersionId        13192 non-null  object 
 8   calcFeature           13192 non-null  object 
 9   subscription_name     13192 non-null  object 
 10  message_id            13192 non-null  object 
 11  publish_time          13192 non-null  object 
 12  attributes            13192 non-null  object 
 13  trenchCategory        13192 non-null  object 
 14  deviceOs              13192 non-null  object 
dtypes: float64(1), int6

In [152]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e33a613d-2950-4ab4-9613-8af29a45590f>

##### Trench 3

In [153]:
sq = """select 
distinct 
loanmaster.customerId customer_id,
r.digitalLoanAccountId,
r.s_demo_score,
ln_vas_opted_flag,
ln_doc_type_rolled,
ln_industry_new_cat_bin,
ln_marital_status,
ln_age,
ln_education_level,
ln_cnt_dependents,
ln_ref2_type,
ln_loan_level_user_type,
ln_ref1_type,
ln_name_email_match_score,
ln_telconame,
ln_city_cat,
ln_brand_bin,
ln_apply_Is_Weekend,
r.ln_os_type osType
from worktable_data_analysis.sil_beta_demo_all_applied_backscored_20240801_20251015 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where s_demo_score is not null
and trench_category = 'Trench 3'
and IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) 
 -- between '2024-08-01' and '2025-01-31'
 >= '2024-08-01'
 ;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID b83690c2-4871-4421-b585-54b7e085a31f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (13866, 19)


In [154]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 's_demo_score',
       'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType'],
      dtype='object')

In [155]:
feature_column = [ 'ln_vas_opted_flag', 'ln_doc_type_rolled', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_age', 'ln_education_level',
       'ln_cnt_dependents', 'ln_ref2_type', 'ln_loan_level_user_type',
       'ln_ref1_type', 'ln_name_email_match_score', 'ln_telconame',
       'ln_city_cat', 'ln_brand_bin', 'ln_apply_Is_Weekend', 'osType']

dfd = transform_datav2(data, feature_column, a='s_demo_score', modelDisplayName='beta_demo_model_sil', tc='Trench 3', subscription_name = 'Cash November 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2457556,bf70f541-c49c-4a21-97a3-035da2a25425,85e2f40b-ff97-4d17-8677-4275a396e9fb,0.091457,2025-12-23T22:34:08.719836,2025-12-23T22:34:08.719836,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,685386eb-7996-4fb1-8ed7-bd5653bd1c00,2025-12-23T22:34:08.719836,{},Trench 3,Android
1,2476364,bef9aa00-8258-47fa-b5f4-3394fe9257d5,3682a67e-9609-4e45-8ae8-92afed64c38f,0.124100,2025-12-23T22:34:08.719836,2025-12-23T22:34:08.719836,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,078bc2e8-562b-47c4-b27f-b2d67e4c483b,2025-12-23T22:34:08.719836,{},Trench 3,Android
2,2468641,a02da15d-1987-4b95-a78c-c98a032fadcf,e68540b7-ab1a-4d9d-8035-db76ad557576,0.169126,2025-12-23T22:34:08.719836,2025-12-23T22:34:08.719836,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,85473e83-2eae-4858-9af1-56b898e0da5f,2025-12-23T22:34:08.719836,{},Trench 3,Android
3,2531402,dee91fba-aafc-4ec7-95ed-c26c38c4fc80,268fe6d3-51bd-43b3-ac91-21b3f9f8370a,0.136102,2025-12-23T22:34:08.719836,2025-12-23T22:34:08.719836,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,0bd1e1b4-bf74-42b8-bb5a-fc6f64fe70c9,2025-12-23T22:34:08.719836,{},Trench 3,Android
4,2436552,f5fa8436-ef4e-4871-9527-25cbda7ed7ae,99e8a9d5-9ef1-466e-979e-f2c4afba6f40,0.109854,2025-12-23T22:34:08.719836,2025-12-23T22:34:08.719836,beta_demo_model_sil,v2,"{""ln_vas_opted_flag"": ""0"", ""ln_doc_type_rolled...",Cash November 25 Models,85b2de7e-9d94-4021-accd-f2520a7ae292,2025-12-23T22:34:08.719836,{},Trench 3,iOS


In [156]:
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

In [157]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13866 entries, 0 to 13865
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            13866 non-null  int64  
 1   digitalLoanAccountId  13866 non-null  object 
 2   crifApplicationId     13866 non-null  object 
 3   prediction            13866 non-null  float64
 4   start_time            13866 non-null  object 
 5   end_time              13866 non-null  object 
 6   modelDisplayName      13866 non-null  object 
 7   modelVersionId        13866 non-null  object 
 8   calcFeature           13866 non-null  object 
 9   subscription_name     13866 non-null  object 
 10  message_id            13866 non-null  object 
 11  publish_time          13866 non-null  object 
 12  attributes            13866 non-null  object 
 13  trenchCategory        13866 non-null  object 
 14  deviceOs              13866 non-null  object 
dtypes: float64(1), int6

In [158]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=74509874-2072-4768-8ff7-5877f8904b04>

# End